In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors
import csv
import numpy as np
import pandas as pd
import cobra

In [2]:
Annotation = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Annotation', index_col=0)
Annotation.index = Annotation.index.map(str)
Annotation = Annotation.fillna('')
Transcriptomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Transcriptomics', header=[0,1,2,3], index_col=0)
Transcriptomics.index = Transcriptomics.index.map(str)
Proteomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Proteomics', header=[0,1,2], index_col=0)
Proteomics.index = Proteomics.index.map(str)
Fitness = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Fitness', index_col=0)
Fitness.index = Fitness.index.map(str)

In [3]:
def background_gradient(s, cmap='seismic', text_color_threshold=0.408):
    lim = max(abs(s.min().min()),abs(s.max().max()))
    rng = 2.0*lim
    norm = colors.Normalize(-lim - (rng * 0.2), lim + (rng * 0.2))
    rgbas = plt.cm.get_cmap(cmap)(norm(s.values))
    def relative_luminance(rgba):
        r, g, b = (x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4) for x in rgba[:3])
        return 0.2126 * r + 0.7152 * g + 0.0722 * b
    def css(rgba):
        dark = relative_luminance(rgba) < text_color_threshold
        text_color = '#f1f1f1' if dark else '#000000'
        return 'background-color: {b};color: {c};'.format(b=colors.rgb2hex(rgba), c=text_color)

    if s.ndim == 1:
        return [css(rgba) for rgba in rgbas]
    else:
        return pd.DataFrame([[css(rgba) for rgba in row] for row in rgbas], index=s.index, columns=s.columns)

def Show_Data(x):
    display(Transcriptomics.loc[x].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Proteomics.index]
    display(Proteomics.loc[temp].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Fitness.index]
    display(Fitness.loc[temp].style.apply(background_gradient, cmap='seismic', axis=None))
    return;

In [4]:
model = cobra.io.load_json_model("IFO0880_GPR_2a.json")

In [5]:
eco = cobra.io.load_json_model('../../Data/BiGG_Models/iML1515.json')
sce = cobra.io.load_json_model('../../Data/BiGG_Models/iMM904.json')
hsa = cobra.io.load_json_model('../../Data/BiGG_Models/RECON1.json')
hsa2 = cobra.io.load_json_model('../../Data/BiGG_Models/Recon3D.json')
ptri = cobra.io.load_json_model('../../Data/BiGG_Models/iLB1027_lipid.json')
ppu = cobra.io.load_json_model('../../Data/BiGG_Models/iJN746.json')

In [6]:
with model:
    sol = model.optimize()
    print(sol)
    print()
    for k, v in sol.shadow_prices.items():
        if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
            print(k, v)

<Solution 0.137 at 0x1033dbbd68>

adp_c -0.4595342375266865
atp_c -0.4708528148056687
amp_c -0.44821566024770426
cmp_c -0.05885660185070177
glu__L_c -0.028673729106749368
gmp_c -0.47462567389865584
ser__L_c -0.15996922554294643
cys__L_c -0.1373320709849902
ala__L_c -0.09658519278065125
ump_c -0.047538024571719545
gln__L_c -0.03999230638573159
dgmp_c -0.4633070966196757
pe_RT_r -264.0141153222689
his__L_c -0.25806356196080116
dtmp_c -0.18487009555670572
trp__L_c -0.22788068921684132
met__L_c -0.12601349370600967
pro__L_c -0.0173551518277677
phe__L_c -0.022637154557965222
tyr__L_c -0.033955731836945335
asp__L_c 1.1102230246251565e-15
ile__L_c -0.012073149097580949
dcmp_c -0.036219447292739376
mannan_r -0.011318577278982224
damp_c -0.4368970829687241
arg__L_c -0.10488548278523885
asn__L_c -0.022637154557963335
glycogen_c -0.022637154557964445
13BDglcn_c -0.022637154557964445
thr__L_c 4.440892098500626e-16
lys__L_c -0.027164585469555635
val__L_c -0.015846008190575023
ergst_r 1.967168731087

In [7]:
for m in model.reactions.get_by_id('BIOMASS_RT').reactants:
    print(m.id, model.reactions.get_by_id('BIOMASS_RT').get_coefficient(m.id))

13BDglcn_c -1.1348
ala__L_c -0.4588
amp_c -0.046
arg__L_c -0.1607
asn__L_c -0.1017
asp__L_c -0.2975
atp_c -59.276
cmp_c -0.0447
cys__L_c -0.0066
damp_c -0.0036
dcmp_c -0.0024
dgmp_c -0.0024
dtmp_c -0.0036
ergst_r -0.0007
gln__L_c -0.1054
glu__L_c -0.3018
gly_c -0.2904
glycogen_c -0.5185
gmp_c -0.046
h2o_c -59.276
his__L_c -0.0663
ile__L_c -0.1927
leu__L_c -0.2964
lys__L_c -0.2862
mannan_r -0.8079
met__L_c -0.0507
pa_RT_r -6e-06
pc_RT_r -6e-05
pe_RT_r -4.5e-05
phe__L_c -0.1339
pro__L_c -0.1647
ps_RT_r -1.7e-05
ptd1ino_RT_r -5.3e-05
ribflv_c -0.00099
ser__L_c -0.1854
so4_c -0.02
thr__L_c -0.1914
tre_c -0.0234
triglyc_RT_r -6.6e-05
trp__L_c -0.0284
tyr__L_c -0.102
ump_c -0.0599
val__L_c -0.2646
zymst_r -0.0015


In [8]:
model.medium

{'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_glc__D_e': 10.0}

In [9]:
# Check if model can produce chitin and other cofactors
# chitin and cofactors is missing in iMM904 biomass
# nmp and dnmp in iMM904, check ntp and dntp biosynthesis
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c',
             'chitin_c','ctp_c','gtp_c','utp_c','datp_c','dctp_c','dgtp_c','dttp_c']
with model:
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234042
coa_c 0.13547854502629142
fad_c 0.13588334572423133
gthrd_c 0.13632034044837388
hemeA_m 0.0
	 hemeA_m -100.0
nad_c 0.13561558122628062
nadp_c 0.13560034129332552
q9_m 0.0
	 q9_m -100.0
thf_c 0.0
	 thf_c -100.0
thmpp_c 0.0
	 thmpp_c -100.0
5mthf_c 0.0
	 5mthf_c -99.99999999999991
btn_m 0.0
	 btn_m -100.0
lipopb_m 0.0
	 lipopb_m -100.0
mlthf_c 0.0
	 mlthf_c -99.99999999999991
ptrc_c 0.136506426473676
pydx5p_c 0.1365311390189468
spmd_c 0.1365774991663941
chitin_c 0.1364848103339987
ctp_c 0.13646628766234234
gtp_c 0.13590171047694038
utp_c 0.13648172287288304
datp_c 0.13595274973450394
dctp_c 0.136497161575454
dgtp_c 0.13591701822992183
dttp_c 0.13629467782836888


Folate biosynthesis (from chorismate via 4-aminobenzoate)  
GTPCI: gtp_c + h2o_c --> ahdt_c + for_c + h_c 10332  
AKP1: ahdt_c + 3.0 h2o_c --> dhnpt_c + 2.0 h_c + 3.0 pi_c 13409  
PTHPS: ahdt_c --> 6pthp_c + pppi_c 10583  
DNTPPA: ahdt_c + h2o_c --> dhpmp_c + h_c + ppi_c 15385  
DNMPPA: dhpmp_c + h2o_c --> dhnpt_c + pi_c 14615 or 14875 or 15385  

DHNPAm: dhnpt_m --> 2ahhmp_m + gcald_m + h_m 14377  
HPPKm: 2ahhmp_m + atp_m --> 2ahhmd_m + amp_m + h_m 14377  
DHPSm: 2ahhmp_m + 4abz_m --> dhpt_m + h2o_m 14377  
FOLD3m: 2ahhmd_m + 4abz_m --> dhpt_m + ppi_m 14377  

DHPS: 2ahhmp_c + 4abz_c --> dhpt_c + h2o_c 14377  
FOLD3: 2ahhmd_c + 4abz_c --> dhpt_c + ppi_c 14377  
FOLD3_1: 2ahhmd_c + 4abz_c --> dhpt_c + h_c + ppi_c 14377  

HPPK_1: 2ahhmp_c + atp_c --> 2ahhmd_c + amp_c 10332  
HPPK2: 6hmhpt_c + atp_c → 6hmhptpp_c + amp_c + h_c  
DHPS2: 4abz_c + 6hmhptpp_c --> dhpt_c + ppi_c 14377  
6hmhpt (E. coli) is same as 2ahhmp (S. cer)  
6hmhptpp (E. coli) is same as 2ahhmd (S. cer)  

DHFS: atp_c + dhpt_c + glu__L_c --> adp_c + dhf_c + h_c + pi_c 10460 or 14803  
DHFR: dhf_c + h_c + nadph_c <=> nadp_c + thf_c 10845  
DHFR2i: dhf_c + h_c + nadh_c --> nad_c + thf_c 10845  

DHFOR: fol_c + h_c + nadh_c --> dhf_c + nad_c 10845  
FOLR2_1: fol_c + h_c + nadph_c --> dhf_c + nadp_c 10845  
FPGS4: 4.0 atp_c + dhf_c + 4.0 glu__L_c --> 5dhf_c + 4.0 adp_c + 4.0 h_c + 4.0 pi_c 14803  
TMDS: dump_c + mlthf_c --> dhf_c + dtmp_c 10845  

FPGS4m: 4.0 atp_m + dhf_m + 4.0 glu__L_m --> 5dhf_m + 4.0 adp_m + 4.0 h_m + 4.0 pi_m 14803

In [10]:
temp = ['10332','13409','15429','14546','8460','8576','13711','9959','11513','13413','9241',
        '10583','15385','14638','12434','14615','14875','14377','10460','14803','10845','12159','15483','9244']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10332,"K01495: GCH1, folE; GTP cyclohydrolase I",,FOL2,GCH1,FOL2,GCH1,Essential,"cysk 13, nucl 4, mito 4, cyto 4, cyto_nucl 4, ...",PQR*
13409,"K01077: E3.1.3.1, phoA, phoB; alkaline phospha...",,PHO8,"ALPI,ALPL,ALPP,ALPPL2",PHO8,ALPL,Not Essential,"plas 14, nucl 4.5, cyto_nucl 4.5, cyto 3.5, mi...",GDF*
15429,"K01077: E3.1.3.1, phoA, phoB; alkaline phospha...",S,,,PHO8,ALPPL2,Not Essential,"extr 17, mito 6, cyto 2, vacu 2",PRH*
14546,K17623: HDHD1; pseudouridine 5'-phosphatase,,YKL033W-A,PUDP,YKL033W-A,PUDP,Not Essential,"cyto 10.5, cysk 9, cyto_nucl 8, nucl 4.5",YDQ*
8460,KOG2914: Predicted haloacid-halidohydrolase an...,,,,DOG2,,Not Essential,"cyto 13.5, cyto_nucl 12.5, nucl 10.5",TIDA
8576,"K19270: yfbT, yniC; sugar-phosphatase",,,,GPP2,,Not Essential,"cyto 18, cysk 8",FVV*
13711,K01835: pgm; phosphoglucomutase,,PRM15,"PGM2,PGM2L1",PRM15,PGM2,Not Essential,"cyto 15.5, cyto_nucl 9, mito 6, pero 2, cysk 2",KPE*
9959,K01835: pgm; phosphoglucomutase,,"PGM1,PGM2","PGM1,PGM5",PGM2,PGM1,Not Essential,"cyto 13.5, cyto_nucl 10, pero 5, nucl 3.5, mito 3",VIT*
11513,HMMPfam:haloacid dehalogenase-like hydrolase:P...,,,,,,Not Essential,"mito 10, cyto 7.5, cyto_nucl 6.5, pero 5, nucl...",AYP*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
13409,0.0829495,0.0345418,0.0179658,0.0812198,0.411433,-0.0471804,0.110333,0.114602,0.275288,0.104641,0.279144
15429,0.25048,0.082651,0.0362311,0.0420706,0.202925,0.132586,0.246786,0.268539,0.135507,0.150893,-0.180486
14546,0.441895,0.052392,0.302713,0.220549,-0.147649,0.167133,-0.101398,-0.225625,-0.193782,-0.238061,0.105055
8460,-0.0791731,-0.223637,0.153426,-0.225143,0.229381,-0.210056,-0.184662,0.201914,-0.0619675,-0.0264,-0.0905085
8576,0.0928334,0.0403074,0.342139,0.0272539,1.10922,0.219086,-0.299253,0.0333091,0.136365,-0.0567877,0.190766
13711,-0.0790543,-0.0552581,0.00927533,-0.070228,-0.0211338,-0.295143,0.100453,0.339881,0.282623,0.339392,-0.0223127
9959,-0.127159,-0.335271,0.0959883,-0.361527,-0.571583,-0.0651405,0.117287,-0.424015,0.209902,-0.174774,-0.293563
11513,0.0450204,0.0302811,-0.04327,0.0507433,-0.535173,0.0529926,-0.0692324,-0.128559,-0.175321,-0.272272,-0.0418747
13413,0.0653619,-0.121519,0.17951,0.0909367,0.399522,0.263557,-0.530393,0.667075,0.651914,0.610799,0.728724


In [11]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

GTPCI: gtp_c + h2o_c --> ahdt_c + for_c + h_c 10332
GTPCIn: gtp_n + h2o_n --> ahdt_n + for_n + h_n 10332

AKP1: ahdt_c + 3.0 h2o_c --> dhnpt_c + 2.0 h_c + 3.0 pi_c 13409
ALKP: dhap_c + h2o_c --> dha_c + pi_c 13409 or 14546

15429 no reactions

ALKP: dhap_c + h2o_c --> dha_c + pi_c 13409 or 14546
E4PP: e4p_c + h2o_c --> erthrs_c + pi_c 14546 or 8460 or 8576
F1PP: f1p_c + h2o_c --> fru_c + pi_c 14546 or 8576
F6PP: f6p_c + h2o_c --> fru_c + pi_c 14546 or 8576
GNP: 6pgc_c + h2o_c --> glcn_c + pi_c 14546
PGMT: g1p_c <=> g6p_c 13711 or 14546 or 8460 or 9959
R5PP: h2o_c + r5p_c --> pi_c + rib__D_c 14546 or 8576

2DOXG6PP: 2doxg6p_c + h2o_c --> 2dglc_c + pi_c 13413 or 8460 or 8576
E4PP: e4p_c + h2o_c --> erthrs_c + pi_c 14546 or 8460 or 8576
GAPP: g3p_c + h2o_c --> glyald_c + pi_c 8460
MN6PP: h2o_c + man6p_c --> man_c + pi_c 8460
PGMT: g1p_c <=> g6p_c 13711 or 14546 or 8460 or 9959

2DOXG6PP: 2doxg6p_c + h2o_c --> 2dglc_c + pi_c 13413 or 8460 or 8576
E4PP: e4p_c + h2o_c --> erthrs_c + pi_c 145

In [12]:
# 13409 is PHO8 -> make nicotinamide riboside from NMN, add NMNHYD from Recon3D
# Also farnesyl diphosphatase, frdp + h2o -> farnesol + ppi
r = hsa2.reactions.get_by_id('NMNHYD').copy()
r.gene_reaction_rule = '13409'
model.add_reactions([r])
# ALKP is by phoA in E. coli, AKP1 is wrong, remove
model.reactions.get_by_id('ALKP').gene_reaction_rule = '13409'
model.remove_reactions(['AKP1'], remove_orphans=True)
# 14546 is pseudouridine 5'-phosphatase, 16646 pseudouridine kinase
# add PSURIK from eco, set gene to 16646
# modify ALKP to PSURIP and set gene to 14546
r = eco.reactions.get_by_id('PSURIK').copy()
r.gene_reaction_rule = '16646'
model.add_reactions([r])
r = model.reactions.get_by_id('ALKP').copy()
r.id = 'PSURIP'
r.name = 'Pseudouridine 5-phosphatase'
r.gene_reaction_rule = '14546'
model.add_reactions([r])
r.add_metabolites({'dhap_c': 1.0, 'dha_c': -1.0, 'psd5p_c': -1.0, 'psuri_c': 1.0})
# 8460 is glycerol-1-phosphatase, also glyc3p
model.reactions.get_by_id('G3PT').gene_reaction_rule = '8460'
# 8576 is sugar phosphatase
model.reactions.get_by_id('E4PP').gene_reaction_rule = '8576'
model.reactions.get_by_id('F1PP').gene_reaction_rule = '8576'
model.reactions.get_by_id('F6PP').gene_reaction_rule = '8576'
model.reactions.get_by_id('GNP').gene_reaction_rule = '8576'
model.reactions.get_by_id('R5PP').gene_reaction_rule = '8576'
# 13413 is 2-deoxyglucose-6-phosphatase
model.reactions.get_by_id('2DOXG6PP').gene_reaction_rule = '13413'
# 13711 and 9959 PGM2 and PGM1
model.reactions.get_by_id('PGMT').gene_reaction_rule = '13711 or 9959'
# 9241 is phosphomannomutase
model.reactions.get_by_id('PMANM').gene_reaction_rule = '9241'
# DNADDP is by 12434 NUDT12 or 14638 NPP/ENPP
# replace NADDPp with NADDP from eco, gene is 12434
model.reactions.get_by_id('DNADDP').gene_reaction_rule = '12434 or 14638'
r = eco.reactions.get_by_id('NADDP').copy()
r.gene_reaction_rule = '12434'
model.add_reactions([r])
model.remove_reactions(['NADDPp'], remove_orphans=True)
# FADDP and UDPGP is okay, by ENPP
# DNTPPA is correct, by 15385 8-oxo-dGTP diphosphatase
# DNMPPA is monophosphate, no gene is known in metacyc, iMM904 gene GET3 is wrong
model.reactions.get_by_id('DNMPPA').gene_reaction_rule = ''
# 14377 FOL1 cytosolic, old reactions are wrong, add DHNPA2r and HPPK2 from eco, keep DHPS2
# remove DHNPAm,DHPS,DHPSm,FOLD3,FOLD3m,HPPKm
r1 = eco.reactions.get_by_id('DHNPA2r').copy()
r1.gene_reaction_rule = '14377'
r2 = eco.reactions.get_by_id('HPPK2').copy()
r2.gene_reaction_rule = '14377'
model.add_reactions([r1,r2])
remove = ['DHNPAm','DHPS','DHPSm','FOLD3','FOLD3m','HPPKm']
model.remove_reactions(remove, remove_orphans=True)
# 10460 is DHFS, 14803 is FPGS
model.reactions.get_by_id('DHFS').gene_reaction_rule = '10460'
model.reactions.get_by_id('THFGLUS').gene_reaction_rule = '14803'
# 10845 is thymidylate synthase, TMDS is correct
# dihydrofolate reductase is 12159 (gene model is wrong, 947542 correct with sigP)
# Add DHFRim from sce, and set gene to 12159 (NADP)
# mlthf reductase is 15483 or 9244
# remove wrong folate reductase reactions
model.reactions.get_by_id('DHFR').gene_reaction_rule = '12159'
model.reactions.get_by_id('FOLR2_1').gene_reaction_rule = '12159'
r = sce.reactions.get_by_id('DHFRim').copy()
r.gene_reaction_rule = '12159'
model.add_reactions([r])
remove = ['DHFOR','DHFR2i','THFOR1','THFOR2']
model.remove_reactions(remove, remove_orphans=True)

In [13]:
# Still need to make thf, check where glycald is going
for r in sorted(model.metabolites.get_by_id('gcald_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('gcald_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

2DDARAA: 2ddara_c <=> gcald_c + pyr_c 12061
DHNPA2r: dhnpt_c <=> 6hmhpt_c + gcald_c 14377
GCALDD: gcald_c + h2o_c + nad_c --> glyclt_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323

GCALDDm: gcald_m + h2o_m + nad_m --> glyclt_m + 2.0 h_m + nadh_m 12042 or 13426


In [14]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('Aldehyde dehydrogenase') |
                        Annotation['Combined Annotations'].str.contains(' aldehyde dehydrogenase') |
                        Annotation['Combined Annotations'].str.contains(' retinal dehydrogenase') ]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10421,KOG2450: Aldehyde dehydrogenase,,,,ALD5,ALDH1,Not Essential,"cyto 19.5, cyto_nucl 14.5, nucl 6.5",RYG*
10657,KOG2454: Betaine aldehyde dehydrogenase,A,MSC7,,MSC7,ALDH1,Not Essential,"plas 15, mito 5, cyto 3, E.R. 2, cyto_pero 2",SAK*
11124,KOG2450: Aldehyde dehydrogenase,,,,ALD4,ALDH1,Not Essential,"cyto 22.5, cyto_nucl 12, mito 2",GAI*
12002,KOG2456: Aldehyde dehydrogenase,,,,,ALDH3,Not Essential,"mito 8, extr 7, cyto 5, mito_nucl 5",FFV*
12042,K00128: ALDH; aldehyde dehydrogenase (NAD+),,"ALD5,ALD4,ALD6,ALD3,ALD2","ALDH1A1,ALDH1A2,ALDH1A3,ALDH1B1,ALDH2",ALD5,ALDH2,Not Essential,"cyto 18.5, cyto_nucl 10, mito 4, pero 4",NPL*
12950,KOG2450: Aldehyde dehydrogenase,,,,UGA2,ALDH9,Not Essential,"cyto 21.5, cyto_nucl 13, mito 3",PSA*
13426,K07249: E1.2.1.36; retinal dehydrogenase,,"ALD5,ALD4,ALD6,ALD3,ALD2","ALDH1A1,ALDH1A2,ALDH1A3,ALDH1B1,ALDH2",ALD5,ALDH1,Not Essential,"mito 25.5, cyto_mito 14",WPL*
15814,K14085: ALDH7A1; aldehyde dehydrogenase family...,S,,ALDH7A1,ALD4,ALDH7,Not Essential,mito 26,TFD*
16323,K00128: ALDH; aldehyde dehydrogenase (NAD+),,HFD1,"ALDH3A1,ALDH3A2,ALDH3B1,ALDH3B2",HFD1,ALDH3,Not Essential,"cyto 10.5, plas 7, cyto_nucl 7, mito 3, nucl 2...",GQA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10421,0.0359244,0.158257,-0.042663,0.144297,0.101933,0.159197,-0.0958431,-0.19259,0.219806,0.0679839,-0.0525745
10657,-0.469803,-0.464668,-0.230016,-0.51407,-0.156593,-0.504589,-0.23201,-0.350682,0.27828,0.223645,-0.472739
11124,-0.225491,0.0952583,-0.0901236,-0.197801,-0.387562,-0.0191805,0.037875,-0.0790279,-0.0604181,-0.154212,0.18216
12002,-0.514533,-0.440842,-0.333472,-0.175652,-0.520258,-0.329939,-0.0362967,0.48597,0.281434,0.230279,0.0668551
12042,0.170666,-0.143043,0.000953096,-0.0230376,-0.521499,0.0146534,0.144875,-0.295284,0.392859,0.454577,-0.242457
12950,0.00596512,-0.171781,0.205573,0.0569485,0.122395,0.160439,0.000604604,0.0892542,0.233256,0.31056,0.190096
13426,-0.226122,-0.181863,-0.169891,-0.0897023,-0.0133507,-0.00809092,-1.1502,-2.84044,-0.85113,-0.458514,-0.0243377
15814,-0.464573,-0.6107,-0.102958,-0.543758,-0.425179,-0.212745,-0.360015,-1.03866,0.403738,-0.219486,-0.625104
16323,0.000210473,-0.0174318,0.0140589,-0.110443,0.00440814,0.0892768,0.155043,0.103889,-0.177274,-0.243731,-0.140368


In [15]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

10421 no reactions

10657 no reactions

11124 no reactions

12002 no reactions

34DHALDD: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323
34DHPLACOX_NADP: 34dhpac_c + h2o_c + nadp_c <=> 34dhpha_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323
3HPADHi: 3hppnl_c + h2o_c + nad_c --> 3hpp_c + 2.0 h_c + nadh_c 12042 or 13426
3M4HDXPAC: 3mox4hpac_c + h2o_c + nad_c <=> 2.0 h_c + homoval_c + nadh_c 12042 or 13426 or 16323
3MOX4HOXPGALDOX: 3m4hpga_c + h2o_c + nad_c --> 3mox4hoxm_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323
3MOX4HOXPGALDOX_NADP: 3m4hpga_c + h2o_c + nadp_c <=> 3mox4hoxm_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323
4HOXPACDOX_NADP: 4hoxpacd_c + h2o_c + nadp_c <=> 4hphac_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323
5HOXINDACTOX: 5hoxindact_c + h2o_c + nad_c --> 5hoxindoa_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323
5HOXINDACTOXm: 5hoxindact_m + h2o_m + nad_m --> 5hoxindoa_m + 2.0 h_m + nadh_m 12042 or 13426
ABOR: 4abutn_m + h2o_m + nadp_m --> 4abut

12042 cyto ALD  
12950 cyto vdh  
13426 mito ALD  
15814 mito 2-aminoadipate semialdehyde  
16323 mito/ER HFD1 hexadecenal and 4-hydroxybenzaldehyde  

In [16]:
temp = ['11650','14700','8666']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11650,"KOG0023: Alcohol dehydrogenase, class V",,,,ADH2,ADH1B,Not Essential,"cyto 9.5, extr 8, mito 6, cyto_nucl 6, nucl 1.5",VIP*
14700,KOG0725: Reductases with broad range of substr...,,,HSD17B14,YMR226C,DECR2,Not Essential,"cyto 14.5, cyto_nucl 11.5, nucl 5.5, pero 3, c...",YLV*
8666,KOG0725: Reductases with broad range of substr...,,,HSD17B14,SPS19,HSD17B,Not Essential,"cyto 18.5, cyto_nucl 12, nucl 2.5, mito 2, ext...",YVH*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11650,0.162296,-0.125414,0.239373,0.0824604,0.133479,0.152742,0.151596,-0.00688522,-0.0872696,-0.226133,0.148191
14700,-0.136366,0.186102,-0.178815,-0.232998,-0.0364499,-0.0311361,0.24188,-0.184481,-0.138316,0.201217,0.511467
8666,0.246518,-0.0531704,0.304517,-0.024053,-0.100515,-0.173468,-0.0249248,-0.343657,0.409518,0.34975,-0.0161461


In [17]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ALDD2y: acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c 11650 or 12042 or 13426 or 14700 or 16323 or 8666
ALDD3y: h2o_c + nadp_c + ppal_c --> 2.0 h_c + nadph_c + ppa_c 11650

ALDD2y: acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c 11650 or 12042 or 13426 or 14700 or 16323 or 8666

ALDD2y: acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c 11650 or 12042 or 13426 or 14700 or 16323 or 8666



11650 Alcohol dehydrogenase patD; Patulin biosynthesis cluster protein D?  
14700 oxidoreductase, short chain dehydrogenase/reductase family, upregulated in p-coumarate  
8666 oxidoreductase

In [18]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if any(x in r.gene_reaction_rule for x in ['12042','13426']):
        if 'c' in r.compartments:
            print('model.reactions.get_by_id(\''+r.id+'\').gene_reaction_rule = \'12042\'')
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if any(x in r.gene_reaction_rule for x in ['12042','13426']):
        if 'm' in r.compartments:
            print('model.reactions.get_by_id(\''+r.id+'\').gene_reaction_rule = \'13426\'')

model.reactions.get_by_id('34DHALDD').gene_reaction_rule = '12042'
model.reactions.get_by_id('34DHPLACOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('3HPADHi').gene_reaction_rule = '12042'
model.reactions.get_by_id('3M4HDXPAC').gene_reaction_rule = '12042'
model.reactions.get_by_id('3MOX4HOXPGALDOX').gene_reaction_rule = '12042'
model.reactions.get_by_id('3MOX4HOXPGALDOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('4HOXPACDOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('5HOXINDACTOX').gene_reaction_rule = '12042'
model.reactions.get_by_id('ABUTD').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD19x_P').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD19xr').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD20x').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD20y').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD2x').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD

In [19]:
model.reactions.get_by_id('34DHALDD').gene_reaction_rule = '12042'
model.reactions.get_by_id('34DHPLACOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('3HPADHi').gene_reaction_rule = '12042'
model.reactions.get_by_id('3M4HDXPAC').gene_reaction_rule = '12042'
model.reactions.get_by_id('3MOX4HOXPGALDOX').gene_reaction_rule = '12042'
model.reactions.get_by_id('3MOX4HOXPGALDOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('4HOXPACDOX_NADP').gene_reaction_rule = '12042'
model.reactions.get_by_id('5HOXINDACTOX').gene_reaction_rule = '12042'
model.reactions.get_by_id('ABUTD').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD19x_P').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD19xr').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD20x').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD20y').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD2x').gene_reaction_rule = '12042'
model.reactions.get_by_id('ALDD2y').gene_reaction_rule = '12042'
model.reactions.get_by_id('AM6SAD').gene_reaction_rule = '12042'
model.reactions.get_by_id('BAMPPALDOX').gene_reaction_rule = '12042'
model.reactions.get_by_id('FTHFDH').gene_reaction_rule = '12042'
model.reactions.get_by_id('GCALDD').gene_reaction_rule = '12042'
model.reactions.get_by_id('GDBTALDH').gene_reaction_rule = '12042'
model.reactions.get_by_id('GGGABADr').gene_reaction_rule = '12042'
model.reactions.get_by_id('GLACO').gene_reaction_rule = '12042'
model.reactions.get_by_id('IMACTD').gene_reaction_rule = '12042'
model.reactions.get_by_id('LCADi').gene_reaction_rule = '12042'
model.reactions.get_by_id('LCADi_D').gene_reaction_rule = '12042'
model.reactions.get_by_id('MACOXO').gene_reaction_rule = '12042'
model.reactions.get_by_id('NABTNO').gene_reaction_rule = '12042'
model.reactions.get_by_id('PYLALDOX').gene_reaction_rule = '12042'

model.reactions.get_by_id('5HOXINDACTOXm').gene_reaction_rule = '13426'
model.reactions.get_by_id('ABOR').gene_reaction_rule = '13426'
model.reactions.get_by_id('ABUTDm').gene_reaction_rule = '13426'
model.reactions.get_by_id('ALDD20xm').gene_reaction_rule = '13426'
model.reactions.get_by_id('ALDD20ym').gene_reaction_rule = '13426'
model.reactions.get_by_id('ALDD2xm').gene_reaction_rule = '13426'
model.reactions.get_by_id('ALDD2ym').gene_reaction_rule = '13426'
model.reactions.get_by_id('BAMPPALDOXm').gene_reaction_rule = '13426'
model.reactions.get_by_id('GCALDDm').gene_reaction_rule = '13426'
model.reactions.get_by_id('GLACOm').gene_reaction_rule = '13426'
model.reactions.get_by_id('IMACTD_m').gene_reaction_rule = '13426'
model.reactions.get_by_id('NABTNOm').gene_reaction_rule = '13426'
model.reactions.get_by_id('PYLALDOXm').gene_reaction_rule = '13426'
model.reactions.get_by_id('TMABDH1_m').gene_reaction_rule = '13426'

model.reactions.get_by_id('ALDD21').gene_reaction_rule = '16323'
model.reactions.get_by_id('COALDDH').gene_reaction_rule = '16323'

In [20]:
for r in sorted(model.genes.get_by_id('12042').reactions, key=lambda x: x.id):
    for m in r.reactants:
        if not any(x == m.id for x in ['h2o_c','nad_c','nadp_c','h2o_m','nad_m','nadp_m']):
            print(m.id)
            for r2 in m.reactions:
                print(r2, r2.gene_reaction_rule)
            print()

34dhpac_c
34DHALDD: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c 12042
42A12BOOX: dopa_c + h2o_c + o2_c --> 34dhpac_c + h2o2_c + nh4_c 10308 or 13959
34DHPLACOX_NADP: 34dhpac_c + h2o_c + nadp_c <=> 34dhpha_c + 2.0 h_c + nadph_c 12042

34dhpac_c
34DHALDD: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c 12042
42A12BOOX: dopa_c + h2o_c + o2_c --> 34dhpac_c + h2o2_c + nh4_c 10308 or 13959
34DHPLACOX_NADP: 34dhpac_c + h2o_c + nadp_c <=> 34dhpha_c + 2.0 h_c + nadph_c 12042

3hppnl_c
3HPADHi: 3hppnl_c + h2o_c + nad_c --> 3hpp_c + 2.0 h_c + nadh_c 12042

3mox4hpac_c
3M4HDXPAC: 3mox4hpac_c + h2o_c + nad_c <=> 2.0 h_c + homoval_c + nadh_c 12042

3m4hpga_c
3MOX4HOXPGALDOX: 3m4hpga_c + h2o_c + nad_c --> 3mox4hoxm_c + 2.0 h_c + nadh_c 12042
3MOX4HOXPGALDOX_NADP: 3m4hpga_c + h2o_c + nadp_c <=> 3mox4hoxm_c + 2.0 h_c + nadph_c 12042

3m4hpga_c
3MOX4HOXPGALDOX: 3m4hpga_c + h2o_c + nad_c --> 3mox4hoxm_c + 2.0 h_c + nadh_c 12042
3MOX4HOXPGALDOX_NADP: 3m4hpga_c + h2o_c + nadp_c <=

In [21]:
# 3MOX4HOXPGALDOX metanephrine degradation in human
# 5HOXINDACTOX serotonin degradation in human
# GGGABADr/GDBTALDH bacterial putrescine/arginine degradation
# remove blocked reactions 3HPADHi, 3M4HDXPAC, GLACO, NABTNO
remove = ['3MOX4HOXPGALDOX','3MOX4HOXPGALDOX_NADP','5HOXINDACTOX','GGGABADr','GDBTALDH','3HPADHi','3M4HDXPAC','GLACO',
          'NABTNO','PYLALDOX']
model.remove_reactions(remove, remove_orphans=True)

In [22]:
# GDBTALDH is connected to amidase reaction
for r in sorted(model.genes.get_by_id('10276').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

AMID: 4gudbd_c + h2o_c --> 4gudbutn_c + nh4_c 10276 or 10277 or 12540 or 12553 or 12640 or 12842
AMID2: h2o_c + pad_c --> nh4_c + pac_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791
AMID3: h2o_c + iad_c --> ind3ac_c + nh4_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791


In [23]:
temp = ['10276','10277','12540','12553','12842','13791','12161','12640','11436','13380','9167','12123']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10276,"K01426: E3.5.1.4, amiE; amidase",,AMD2,,AMD2,FAAH,Not Essential,"cyto 13.5, cyto_nucl 10.5, nucl 4.5, mito 3, p...",WAE*
10277,"K01426: E3.5.1.4, amiE; amidase",,AMD2,,AMD2,FAAH,Not Essential,"mito 14, cyto 9, nucl 2, pero 2",WAA*
12540,KOG1212: Amidases,,AMD2,,AMD2,FAAH,Not Essential,"mito 10, cysk 6, cyto 5, pero 3, extr 2",QSA*
12553,KOG1212: Amidases,S,AMD2,,AMD2,FAAH,Not Essential,mito 27,QSA*
12842,"K01426: E3.5.1.4, amiE; amidase",,AMD2,,AMD2,FAAH,Not Essential,"cyto 13.5, cyto_nucl 10, nucl 5.5, pero 5",KKA*
13791,"K01426: E3.5.1.4, amiE; amidase",,,,AMD2,FAAH,Not Essential,"cyto 13.5, cyto_mito 11.5, mito 8.5, nucl 2, p...",EEQ*
12161,K15528: FAAH; fatty acid amide hydrolase,,,FAAH,AMD2,FAAH,Not Essential,"cyto 11, mito 10, nucl 3, pero 2",RVY*
12640,"K02433: gatA, QRSL1; aspartyl-tRNA(Asn)/glutam...",,HER2,QRSL1,HER2,QRSL1,Essential,mito 25,KAE*
11436,"K02434: gatB, PET112; aspartyl-tRNA(Asn)/gluta...",S,PET112,GATB,PET112,GATB,Not Essential,"mito 24, cyto 3",LGV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10276,0.0590597,-0.183785,0.0217332,0.217737,0.0962688,0.400174,-0.602794,0.20458,-0.0993253,-0.246385,0.0940215
10277,-0.0317559,-0.00631653,-0.182822,-0.0983566,-0.30335,0.0500607,-0.0843293,-0.292154,-0.291562,-0.141471,0.55294
12553,-0.0449658,-0.0975784,-0.208307,-0.150953,0.248676,-0.226523,0.18321,0.155096,0.137944,-0.184634,0.0300694
12842,-0.0326962,0.0985683,0.114314,0.138411,0.448384,0.230723,0.456779,-0.0532847,0.146205,0.0458719,-0.0169437
13791,0.20527,0.0745708,0.0792896,0.103773,0.000193625,0.194923,0.216039,0.139521,-0.0856743,-0.17181,0.102706
12161,-0.266226,-0.209809,-0.279097,-0.156056,-0.336039,-0.0991283,-0.226048,0.175646,-0.254703,0.0193068,0.345903
11436,-0.125742,-0.00340533,-0.101468,0.00454544,-0.101642,-0.139944,0.119935,0.000208259,0.0940378,0.207925,-0.528785
13380,0.212952,0.48865,0.134225,0.480758,-0.595638,0.105697,-0.289997,-0.915948,0.122397,-0.08248,-0.750243


In [24]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

AMID: 4gudbd_c + h2o_c --> 4gudbutn_c + nh4_c 10276 or 10277 or 12540 or 12553 or 12640 or 12842
AMID2: h2o_c + pad_c --> nh4_c + pac_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791
AMID3: h2o_c + iad_c --> ind3ac_c + nh4_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791

AMID: 4gudbd_c + h2o_c --> 4gudbutn_c + nh4_c 10276 or 10277 or 12540 or 12553 or 12640 or 12842
AMID2: h2o_c + pad_c --> nh4_c + pac_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791
AMID3: h2o_c + iad_c --> ind3ac_c + nh4_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791

AMID: 4gudbd_c + h2o_c --> 4gudbutn_c + nh4_c 10276 or 10277 or 12540 or 12553 or 12640 or 12842
AMID2: h2o_c + pad_c --> nh4_c + pac_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791
AMID3: h2o_c + iad_c --> ind3ac_c + nh4_c 10276 or 10277 or 12161 or 12540 or 12553 or 12842 or 13791

AMID: 4gudbd_c + h2o_c --> 4gudbutn_c + nh4_c 10276 or 10277 or 12540 or 12553 or 12640 or 12842
AMID2:

In [25]:
# 12161 is fatty acid amide hydrolase
m = model.metabolites.get_by_id('ocdcea_c').copy()
m.id = 'ocdcead_c'
m.name = 'Oleamide'
m.formula = 'C18H35NO'
m.charge = 0
model.add_metabolites([m])
r = model.reactions.get_by_id('AMID').copy()
r.id = 'FAAH'
r.name = 'Fatty acid amide hydrolase'
r.gene_reaction_rule = '12161'
model.add_reactions([r])
r.add_metabolites({'4gudbd_c': 1.0, '4gudbutn_c': -1.0, 'ocdcead_c': -1.0, 'ocdcea_c': 1.0})
# 12640 and 11436 are aspartyl-tRNA(Asn)/glutamyl-tRNA(Gln) amidotransferase subunits gatA and gatB
# gatC is missing -> check tRNA synthesis later 
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4301749/
model.reactions.get_by_id('AMID').gene_reaction_rule = '10276 or 10277 or 12540 or 12553 or 12842 or 13791'
model.reactions.get_by_id('AMID2').gene_reaction_rule = '10276 or 10277 or 12540 or 12553 or 12842 or 13791'
model.reactions.get_by_id('AMID3').gene_reaction_rule = '10276 or 10277 or 12540 or 12553 or 12842 or 13791'
model.reactions.get_by_id('AMID_1').gene_reaction_rule = '10276 or 10277 or 12540 or 12553 or 12842 or 13791'

In [26]:
for r in sorted(model.genes.get_by_id('13426').reactions, key=lambda x: x.id):
    for m in r.reactants:
        if not any(x == m.id for x in ['h2o_c','nad_c','nadp_c','h2o_m','nad_m','nadp_m']):
            print(m.id)
            for r2 in m.reactions:
                print(r2, r2.gene_reaction_rule)
            print()

5hoxindact_m
5HOXINDACTOXm: 5hoxindact_m + h2o_m + nad_m --> 5hoxindoa_m + 2.0 h_m + nadh_m 13426

4abutn_m
ABUTDm: 4abutn_m + h2o_m + nad_m --> 4abut_m + 2.0 h_m + nadh_m 13426
ABOR: 4abutn_m + h2o_m + nadp_m --> 4abut_m + 2.0 h_m + nadph_m 13426

4abutn_m
ABUTDm: 4abutn_m + h2o_m + nad_m --> 4abut_m + 2.0 h_m + nadh_m 13426
ABOR: 4abutn_m + h2o_m + nadp_m --> 4abut_m + 2.0 h_m + nadph_m 13426

id3acald_m
ALDD20xm: h2o_m + id3acald_m + nad_m --> 2.0 h_m + ind3ac_m + nadh_m 13426
ALDD20ym: h2o_m + id3acald_m + nadp_m --> 2.0 h_m + ind3ac_m + nadph_m 13426

id3acald_m
ALDD20xm: h2o_m + id3acald_m + nad_m --> 2.0 h_m + ind3ac_m + nadh_m 13426
ALDD20ym: h2o_m + id3acald_m + nadp_m --> 2.0 h_m + ind3ac_m + nadph_m 13426

acald_m
ALDD2xm: acald_m + h2o_m + nad_m --> ac_m + 2.0 h_m + nadh_m 13426
ALDD2ym: acald_m + h2o_m + nadp_m --> ac_m + 2.0 h_m + nadph_m 13426
ACALDtm: acald_m <=> acald_c 

acald_m
ALDD2xm: acald_m + h2o_m + nad_m --> ac_m + 2.0 h_m + nadh_m 13426
ALDD2ym: acald_m + h2o_

In [27]:
# 5HOXINDACTOXm serotonin degradation in human
# 4abutn is produced only in cyto, remove ABOR and ABUTDm
# id3acald is produced only in cyto, remove ALDD20ym and ALDD20xm
# bamppald is produced only in cyto, remove BAMPPALDOXm
# similar reasons, remove GCALDDm, GLACOm, IMACTD_m, NABTNOm, PYLALDOXm
remove = ['5HOXINDACTOXm','ABOR','ABUTDm','ALDD20xm','ALDD20ym','BAMPPALDOXm','GCALDDm','GLACOm','IMACTD_m','NABTNOm',
          'PYLALDOXm']
model.remove_reactions(remove, remove_orphans=True)

In [28]:
for r in model.reactions:
    if r.gene_reaction_rule == '16323':
        print(r, r.gene_reaction_rule)

COALDDH: conialdh_c + h2o_c + nad_c --> fer_c + 2.0 h_c + nadh_c 16323
ALDD21: h2o_c + nad_c + pristanal_c --> 2.0 h_c + nadh_c + prist_c 16323


In [29]:
temp = ['16323','12950']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16323,K00128: ALDH; aldehyde dehydrogenase (NAD+),,HFD1,"ALDH3A1,ALDH3A2,ALDH3B1,ALDH3B2",HFD1,ALDH3,Not Essential,"cyto 10.5, plas 7, cyto_nucl 7, mito 3, nucl 2...",GQA*
12950,KOG2450: Aldehyde dehydrogenase,,,,UGA2,ALDH9,Not Essential,"cyto 21.5, cyto_nucl 13, mito 3",PSA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16323,0.000210473,-0.0174318,0.0140589,-0.110443,0.00440814,0.0892768,0.155043,0.103889,-0.177274,-0.243731,-0.140368
12950,0.00596512,-0.171781,0.205573,0.0569485,0.122395,0.160439,0.000604604,0.0892542,0.233256,0.31056,0.190096


In [30]:
Annotation.loc['12950','WolfPSort']

'cyto 21.5, cyto_nucl 13, mito 3'

16323 HFD1 is ER hexadecenal and mito 4-hydroxybenzaldehyde dehydrogenase with nadh  
https://www.sciencedirect.com/science/article/pii/S1097276512003954  
https://www.sciencedirect.com/science/article/pii/S2451945616302902  

hexadecenal (hxdceal) is generated from sphingosine phosphate lyase  
if coming from cer4_18_r, Sphingosine:n-C18:0OH, Cer(d18:1(4E)/18:0(2OH))  
we get a sphingosine (sphings) and a 2-hydroxyhexadecanoate  
if coming from desaturated version of cer1_18_r, Sphingosine:n-C18:0, Cer(d18:1(4E)/18:0)  
we get a sphingosine (sphings) and a normal hydroxy fatty acid  
sphingosine 1-phosphate is degraded to ehamp and hexadecenal  
2-hydroxy fatty acid needs alpha-oxidation  
hexadecenal is oxidized to hexadecenoate, and activated to hdd2coa, enters beta-oxidation

phytosphingosine degradation results in a 2-hydroxy fatty aldehyde,  
which can be oxidized to 2-hydroxy fatty acid by HFD1/ALDH3A2  
fatty acid alpha-oxydation happens in E.R. in human, by HACL2 (not found in Rhodo) and PECR (10912?)  
https://www.pnas.org/content/114/13/E2616  

12950 bacterial vdh can also oxidize 4-hydroxybenzaldehyde, weak sigP, alternative splicing?  
Assign to cyto reactions for now

In [31]:
r = ptri.reactions.get_by_id('ALDD16').copy()
r.gene_reaction_rule = '16323'
model.add_reactions([r])

r = ptri.reactions.get_by_id('ALDD3C161_c').copy()
r.id = 'ALDD3C161'
r.gene_reaction_rule = '16323'
model.add_reactions([r])

for x in ['ALDD21','ALDD16','ALDD3C161']:
    r = model.reactions.get_by_id(x)
    r.id = r.id + 'er'
    for m in r.metabolites:
        if not m.id.replace('_c','_r') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_r')
            m2.compartment = 'r'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})

r = model.reactions.get_by_id('VNDH_2').copy()
r.id = '4HBALDDm'
r.gene_reaction_rule = '16323'
model.add_reactions([r])
for m in r.metabolites:
    if not m.id.replace('_c','_m') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_m')
        m2.compartment = 'm'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})

for x in ['COALDDH','VNDH','VNDH_2','VNDH_3']:
    r = model.reactions.get_by_id(x)
    r.gene_reaction_rule = '12950'

In [32]:
for r in sorted(model.genes.get_by_id('16323').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12950').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

4HBALDDm: 4hbald_m + h2o_m + nad_m --> 4hbz_m + 2.0 h_m + nadh_m 16323
ALDD16er: h2o_r + hxdcal_r + nad_r --> 2.0 h_r + hdca_r + nadh_r 16323
ALDD21er: h2o_r + nad_r + pristanal_r --> 2.0 h_r + nadh_r + prist_r 16323
ALDD3C161er: h2o_r + hxdceal_r + nad_r --> 2.0 h_r + hdc2ea_r + nadh_r 16323

COALDDH: conialdh_c + h2o_c + nad_c --> fer_c + 2.0 h_c + nadh_c 12950
VNDH: h2o_c + nad_c + vanln_c --> 2.0 h_c + nadh_c + vanlt_c 12950
VNDH_2: 4hbald_c + h2o_c + nad_c --> 4hbz_c + 2.0 h_c + nadh_c 12950
VNDH_3: 34dhbald_c + h2o_c + nad_c --> 34dhbz_c + 2.0 h_c + nadh_c 12950


glycolate is oxidized to glyoxylate in peroxisome by 2-hydroxy acid oxidase

In [33]:
for r in sorted(model.metabolites.get_by_id('glyclt_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('glyclt_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('glyclt_x').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

GCALDD: gcald_c + h2o_c + nad_c --> glyclt_c + 2.0 h_c + nadh_c 12042
GLYCLTDy: glx_c + h_c + nadph_c --> glyclt_c + nadp_c 11036 or 9515 or 9520

GLYCLTDxm: glx_m + h_m + nadh_m --> glyclt_m + nad_m 12051

GLYCLTDyp: glx_x + h_x + nadph_x --> glyclt_x + nadp_x 11036
GLYCTO1p: glyclt_x + o2_x --> glx_x + h2o2_x 14950 or 16607


In [34]:
temp = ['EX_glyclt_e','GLYCLTt','GLYCLTtm','GLYCLTtp']
for x in temp:
    r = hsa2.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = ''
    model.add_reactions([r])
model.reactions.get_by_id('EX_glyclt_e').lower_bound = 0.0

In [35]:
for r in sorted(model.metabolites.get_by_id('glx_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('glx_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('glx_x').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

AGTi: ala__L_c + glx_c --> gly_c + pyr_c 8819
GLYCLTDy: glx_c + h_c + nadph_c --> glyclt_c + nadp_c 11036 or 9515 or 9520
UGLYCH: h2o_c + 2.0 h_c + urdglyc_c --> co2_c + glx_c + 2.0 nh4_c 14724

AGTim: ala__L_m + glx_m --> gly_m + pyr_m 8819
DDPGAm: 4h2oglt_m <=> glx_m + pyr_m 12061
GLYCLTDxm: glx_m + h_m + nadh_m --> glyclt_m + nad_m 12051

GLXO2p: glx_x + h2o_x + o2_x --> h2o2_x + h_x + oxa_x 14950 or 16607
GLYCLTDyp: glx_x + h_x + nadph_x --> glyclt_x + nadp_x 11036
GLYCTO1p: glyclt_x + o2_x --> glx_x + h2o2_x 14950 or 16607
GLYOp: gly_x + h2o_x + o2_x --> glx_x + h2o2_x + nh4_x 15994
ICLp: icit_x --> glx_x + succ_x 14022
MALSp: accoa_x + glx_x + h2o_x --> coa_x + h_x + mal__L_x 9457


In [36]:
temp = ['EX_glx_e','GLXt','GLXtm','GLXtp']
for x in temp:
    r = hsa2.reactions.get_by_id(x).copy()
    r.gene_reaction_rule = ''
    model.add_reactions([r])
model.reactions.get_by_id('EX_glx_e').lower_bound = 0.0

In [37]:
# check if model can produce cofactors
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
with model:
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234803
coa_c 0.1354785450262918
fad_c 0.1358833457242384
gthrd_c 0.13632034044837113
hemeA_m 0.0
	 hemeA_m -100.0
nad_c 0.1356155812262751
nadp_c 0.13560034129332388
q9_m 0.0
	 q9_m -100.0
thf_c 0.13597215470566093
thmpp_c 0.0
	 thmpp_c -100.0
5mthf_c 0.13577122443666576
btn_m 0.0
	 btn_m -100.0
lipopb_m 0.0
	 lipopb_m -100.0
mlthf_c 0.1357559495044811
ptrc_c 0.13650642647367617
pydx5p_c 0.13653113901894953
spmd_c 0.13657749916639828


Thiamine biosynthesis  
HETZK: 4mhetz_c + atp_c --> 4mpetz_c + adp_c + h_c 16213  
TMPPP: 2mahmp_c + 4mpetz_c + h_c --> ppi_c + thmmp_c 16213  
TMDPK: atp_c + thm_c --> amp_c + h_c + thmpp_c 16484  
TMDPPK: atp_c + thmpp_c --> adp_c + thmtp_c 16484

Need to add TDP by 14465  
TDP: h2o_c + thmpp_c --> h_c + pi_c + thmmp_c  
Need to add THMP by thiamine phosphate phosphatase (?)  
THMP: h2o_c + thmmp_c --> pi_c + thm_c

2mahmp_c  
Need to add TMN by THI20 12778  
TMN: h2o_c + thm_c --> 4ahmmp_c + 4mhetz_c + h_c missing  
HMPK1: 4ahmmp_c + atp_c --> 4ampm_c + adp_c + h_c 12778  
PMPK: 4ampm_c + atp_c --> 2mahmp_c + adp_c 12778  
There are several more reactions by 12778 THI20 that can synthesis precursors

4mpetz_c
synthesis reactions are hypothetical by 16324 THI4  
gly + [protein]-cys__L + nad -> ADP-5-ethyl-4-methylthiazole-2-carboxylate + [protein]-2amac + 2 h + 3 h2o + ncam  
ADP-5-ethyl-4-methylthiazole-2-carboxylate + h2o -> 4mpetz + h + co2 + AMP  
make a new overall reaction THZPSN4  
gly + cys__L + nad -> 4mpetz + 2amac + co2 + amp + ncam + 3 h + 2 h2o  

2ahmmp_c (from 4ahmmp_c or 4ampm_c)  
there are bacterial synthesis reactions (from air) in BiGG  
AHMMPS: air_c + 2.0 h_c → 4ahmmp_c + gcald_c + pi_c (iMM904)  
AMPMS2: air_c + h2o_c + nad_c → 4ampm_c + 2.0 for_c + 3.0 h_c + nadh_c (iML1515)  
AMPMS3: air_c + amet_c ⇌ 4ampm_c + dad_5_c + for_c + 2.0 h_c + met__L_c + co_c (iJB785)  
Fungal synthesis by HMP-P synthase THI5 is very recent discovery  
https://www.ncbi.nlm.nih.gov/pubmed/22568620  
https://oaktrust.library.tamu.edu/handle/1969.1/155385  
Create a new reaction HMPPS  
pdx5p + his-L + 2 o2 + h2o2 + 2 fe3 -> 4ampm + 4oglu + glx + co2 + 2 fe2 + 4 h

In [38]:
temp = ['16213','16484','12778','16324','11147','14465']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16213,K14154: THI6; thiamine-phosphate diphosphoryla...,,THI6,,THI6,,Not Essential,"mito 11, cyto 7, extr 4, cyto_nucl 4",QRV*
16484,"K00949: thiN, TPK1, THI80; thiamine pyrophosph...",,THI80,TPK1,THI80,TPK1,Essential,"nucl 10.5, cyto_nucl 10, cyto 8.5, mito 5",GAE*
12778,K00877: THI20; hydroxymethylpyrimidine/phospho...,,"THI20,THI21,THI22",,THI20,,Not Essential,"cyto 15.5, cyto_nucl 8.5, mito 8",MPQ*
16324,"K03146: THI4, THI1; thiamine thiazole synthase",,THI4,,THI4,,Not Essential,"cyto 20, mito 7",ISA*
11147,K18278: THI5; pyrimidine precursor biosynthesi...,,"THI13,YFL058W,YJR156C,THI12",,THI5,,Not Essential,"cyto 14, E.R. 5, mito 3, extr 3",VAA*
14465,KOG4313: Thiamine pyrophosphokinase,,YJR142W,,YJR142W,,Not Essential,"cyto 8, mito 7, cyto_nucl 6.333, cyto_pero 6.3...",GPA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16213,-0.296936,-0.341591,-0.0041986,-0.184173,0.0422894,-0.124728,-0.00794017,-0.0866272,0.142589,0.226911,0.0872267
16484,-0.833702,-2.52844,-1.67896,-0.683333,-4.02708,-2.29738,-0.947317,-5.03069,-2.83414,-2.67902,0.361126
12778,0.0606635,0.189572,0.155593,0.30207,0.344105,0.0254225,0.372177,-0.114837,-0.202211,-0.113041,0.0202546
16324,0.186695,-0.0225794,0.00910147,0.144733,0.221646,-0.0879462,0.23013,-0.296677,0.210096,0.205255,0.164019
11147,-0.106836,-0.173124,-0.204399,-0.0157074,-0.273489,-0.327768,0.176538,0.136414,-0.530394,-0.165471,-0.378923
14465,0.350408,0.211929,0.157372,0.0222189,0.572482,-0.0365795,0.16677,0.293391,0.178531,0.416711,-0.0397786


In [39]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

HETZK: 4mhetz_c + atp_c --> 4mpetz_c + adp_c + h_c 16213
TMPPP: 2mahmp_c + 4mpetz_c + h_c --> ppi_c + thmmp_c 16213

TMDPK: atp_c + thm_c --> amp_c + h_c + thmpp_c 16484
TMDPPK: atp_c + thmpp_c --> adp_c + thmtp_c 16484

HMPK1: 4ahmmp_c + atp_c --> 4ampm_c + adp_c + h_c 12778
PMPK: 4ampm_c + atp_c --> 2mahmp_c + adp_c 12778

16324 no reactions

11147 no reactions

14465 no reactions



In [40]:
# 16484 is pyrophosphokinase, remove triphosphate
model.remove_reactions(['TMDPPK'], remove_orphans=True)
# 14465 is Thiamin pyrophosphatase, not kinase - has Nudix hydrolase domain
r = eco.reactions.get_by_id('TDP').copy()
r.gene_reaction_rule = '14465'
model.add_reactions([r])
# Add thiaminase reaction by 12778
r = sce.reactions.get_by_id('TMN').copy()
r.gene_reaction_rule = '12778'
model.add_reactions([r])
# Add thiazole phosphate synthesis reaction by 16324 THI4
r = sce.reactions.get_by_id('THZPSN2_SC').copy()
r.id = 'THZPSN4'
r.name = 'Thiamine thiazole synthase'
r.gene_reaction_rule = '16324'
model.add_reactions([r])
r.add_metabolites({'achms_c': 1.0, 'r5p_c': 1.0, '4abut_c': -1.0, 'ac_c': -1.0, 'nh4_c': -1.0, 'pyr_c': -1.0,
                   'nad_c': -1.0, 'h_c': 4.0, '2amac_c': 1.0, 'amp_c': 1.0, 'ncam_c': 1.0, 'h2o_c': -1.0})
# Add HMP-P synthase reaction by 11147 THI5 and 2-oxo-L-glutamate demand
m = cobra.Metabolite('4oglu_c', name='4-oxo-L-glutamate', formula='C5H6NO5', charge=-1, compartment='c')
model.add_metabolites([m])
r = cobra.Reaction('HMPPS', name='4-amino-5-hydroxymethyl-2-methylpyrimidine phosphate synthase')
r.gene_reaction_rule = '11147'
model.add_reactions([r])
r.add_metabolites({'pdx5p_c': -1.0, 'his__L_c': -1.0, 'o2_c': -2.0, 'h2o2_c': -1.0, 'fe3_c': -4.0,
                   '4ampm_c': 1.0, '4oglu_c': 1.0, 'glx_c': 1.0, 'co2_c': 1.0, 'fe2_c': 4.0, 'h_c': 6.0})
r = cobra.Reaction('DM_4oglu_c', name='4-oxo-L-glutamate demand')
model.add_reactions([r])
r.add_metabolites({'4oglu_c': -1.0})

In [41]:
temp = ['10885','13856','13935','9324','10359','10921','11871','12902','13969']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10885,K01078: E3.1.3.2; acid phosphatase,A,,,DIA3,,Not Essential,"mito 9, plas 6, cyto 4.5, cyto_nucl 3.5, pero ...",PHS*
13856,K01078: E3.1.3.2; acid phosphatase,S,,,,,Not Essential,extr 27,ALL*
13935,K14394: ACP1; low molecular weight phosphotyro...,S,LTP1,ACP1,LTP1,ACP1,Not Essential,"mito 10, cyto 9.5, cyto_nucl 7.5, pero 3, nucl...",GKL*
9324,KOG2466: Uridine permease/thiamine transporter...,,,,DAL4,,Not Essential,plas 27,VAV*
10359,K03457: TC.NCS1; nucleobase:cation symporter-1...,,,,FUI1,,Not Essential,plas 26,VHA*
10921,KOG2466: Uridine permease/thiamine transporter...,A,,,FUR4,,Not Essential,plas 26,EHA*
11871,K03457: TC.NCS1; nucleobase:cation symporter-1...,,,,DAL4,,Not Essential,plas 27,VAV*
12902,K03457: TC.NCS1; nucleobase:cation symporter-1...,A,"FUI1,FUR4,DAL4",,DAL4,,Not Essential,plas 27,ELV*
13969,"K15014: SLC29A1_2_3, ENT1_2_3; solute carrier ...",,FUN26,SLC29A3,,SLC29,Not Essential,"plas 17, E.R. 8",PFV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10885,-0.067265,-0.106314,0.0205561,-0.0456077,-0.324245,0.0817338,-0.0342517,0.178912,-0.00788415,-0.039932,-0.030814
13856,0.0352738,0.140121,0.0520424,-0.219534,-0.195112,-0.308844,0.222728,-0.16456,0.125953,-0.0656023,0.542956
13935,-0.0733152,-0.0910469,0.177923,0.0527015,0.639309,-0.0737553,-0.126422,-0.201468,-0.305069,-0.392733,-0.1807
9324,0.143381,0.177886,0.137487,0.276363,0.211538,-0.0416871,-0.0968904,0.137922,0.239205,-0.0403884,0.140725
10359,-0.0718355,0.205283,-0.276995,0.00506491,0.481236,-0.246722,0.455706,0.582832,-0.121553,-0.101068,0.118733
10921,0.0884597,0.127497,0.0667718,0.118477,0.227692,0.434115,-0.34908,-0.0758182,-0.140076,-0.193443,0.172072
11871,0.00466906,-0.159493,-0.121645,-0.124357,-0.127033,-0.129729,0.26642,0.155626,0.289326,0.181374,-0.510964
12902,0.157584,0.274742,-0.026447,0.145726,0.0343194,-0.0428231,0.00108723,0.310765,0.356296,0.0756319,-0.146704
13969,-0.0391677,-0.338096,-0.0310784,-0.20435,0.209895,-0.0194941,0.0241269,0.533762,0.189846,0.301947,0.461926


In [42]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

ACP1e: fmn_e + h2o_e --> pi_e + ribflv_e 10885 or 13856 or 13935
THMDPe: 2.0 h2o_e + thmpp_e --> h_e + 2.0 pi_e + thm_e 10885
THMPe: h2o_e + thmmp_e --> pi_e + thm_e 10885

ACP1e: fmn_e + h2o_e --> pi_e + ribflv_e 10885 or 13856 or 13935

ACP1_FMN: fmn_c + h2o_c --> pi_c + ribflv_c 13935
ACP1e: fmn_e + h2o_e --> pi_e + ribflv_e 10885 or 13856 or 13935

9324 no reactions

10359 no reactions

ALLTNti: alltn_e --> alltn_c 10921 or 11871 or 12902
THMt2: h_e + thm_e --> h_c + thm_c 10921 or 11871 or 12902
URAt2: h_e + ura_e --> h_c + ura_c 10921 or 11871 or 12902
URIt2: h_e + uri_e --> h_c + uri_c 10921 or 11871 or 12902 or 13969 or (10921 and 13969) or (11871 and 13969) or (12902 and 13969)

ALLTNti: alltn_e --> alltn_c 10921 or 11871 or 12902
THMt2: h_e + thm_e --> h_c + thm_c 10921 or 11871 or 12902
URAt2: h_e + ura_e --> h_c + ura_c 10921 or 11871 or 12902
URIt2: h_e + uri_e --> h_c + uri_c 10921 or 11871 or 12902 or 13969 or (10921 and 13969) or (11871 and 13969) or (12902 and 13969)



In [43]:
# 10885 myo-inositol-hexakisphosphate 3-phosphohydrolase
# Phytic acid + H2O <=> D-myo-Inositol 1,2,4,5,6-pentakisphosphate + Orthophosphate
# No reaction in BiGG, remove 10885 from other reactions
model.remove_reactions(['THMDPe'], remove_orphans=True)
# 13856 acid phosphatase (extra), and 13935 low molecular weight phosphotyrosine protein phosphatase (cyto?)
# FMN + H2O <=> Riboflavin + Orthophosphate
# Thiamin monophosphate + H2O <=> Thiamine + Orthophosphate
# 4-Nitrophenyl phosphate + H2O <=> 4-Nitrophenol + Orthophosphate
# 1-Acyl-sn-glycerol 3-phosphate + H2O <=> 1-Acylglycerol + Orthophosphate
r = sce.reactions.get_by_id('THMP').copy()
r.gene_reaction_rule = '13935'
model.add_reactions([r])
model.reactions.get_by_id('ACP1e').gene_reaction_rule = '13856'
model.reactions.get_by_id('THMPe').gene_reaction_rule = '13856'
# 9324 is similar to 10921 -> add, 13969 is SLC29A3 -> remove
# SLC29A3 mediates both influx and efflux of nucleosides across the membrane (equilibrative transporter).
# Mediates transport of adenine, adenosine and uridine, as well as several nucleoside analog drugs,
# such as anticancer and antiviral agents, including cladribine, cordycepin, tubercidin and AZT. 
# Does not transport hypoxanthine.
model.reactions.get_by_id('ALLTNti').gene_reaction_rule = '9324 or 10921 or 11871 or 12902'
model.reactions.get_by_id('THMt2').gene_reaction_rule = '9324 or 10921 or 11871 or 12902'
model.reactions.get_by_id('URAt2').gene_reaction_rule = '9324 or 10921 or 11871 or 12902'
model.reactions.get_by_id('URIt2').gene_reaction_rule = '9324 or 10921 or 11871 or 12902'

In [81]:
Annotation.loc['10885','WolfPSort']

'mito 9, plas 6, cyto 4.5, cyto_nucl 3.5, pero 2, nucl 1.5, E.R. 1, golg 1, cysk 1, vacu 1'

In [79]:
for r in model.metabolites.minohp_n.reactions:
    print(r, r.gene_reaction_rule)

PMI12346PSn: atp_n + minohp_n --> adp_n + ppmi12346p_n 9763
MI13456PKn: atp_n + mi13456p_n --> adp_n + h_n + minohp_n 15600


In [82]:
for r in model.genes.get_by_id('13260').reactions:
    print(r, r.gene_reaction_rule)

MI1345PKn: atp_n + mi1345p_n --> adp_n + h_n + mi13456p_n 13260
MI145P6Kn: atp_n + mi145p_n --> adp_n + h_n + mi1456p_n 13260
MI145PKn: atp_n + mi145p_n --> adp_n + h_n + mi1345p_n 13260
MI1456PKn: atp_n + mi1456p_n --> adp_n + h_n + mi13456p_n 13260


In [44]:
# check if model can produce cofactors
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
with model:
    model.reactions.get_by_id('FE2t').lower_bound = -1000.0
    model.reactions.get_by_id('FE3t').lower_bound = -1000.0
    model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000.0
    model.reactions.get_by_id('EX_fe3_e').lower_bound = -1000.0
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                #if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                if abs(v) > 1:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234387
coa_c 0.13547854502629672
fad_c 0.13588334572423844
gthrd_c 0.13632034044837513
hemeA_m 0.13525177641251737
nad_c 0.135615581226282
nadp_c 0.13560034129332244
q9_m 0.0
	 4hbz_m -100.0
	 3nphb_m -100.0
	 3npdhb_m -100.0
	 me3dhnpdh_m -100.0
	 2np6mep_m -100.0
	 2np6mobq_m -100.0
	 me2np6mobq_m -100.0
	 2npmhmobq_m -100.0
	 q9_m -100.0
	 q9h2_m -100.0
	 4hbald_m -100.0
thf_c 0.13597215470564794
thmpp_c 0.13561761347615886
5mthf_c 0.13577122443666242
btn_m 0.0
	 btamp_m -100.0
	 btn_m -100.0
	 apoC_Lys_btn_m -100.0
	 dad_5_m 100.0
lipopb_m 0.0
	 dad_5_m 50.0
	 lipoamp_m -100.0
	 lipopb_m -100.0
	 lipoate_m -100.0
mlthf_c 0.13575594950448788
ptrc_c 0.1365064264736814
pydx5p_c 0.13653113901894684
spmd_c 0.13657749916638803


In [45]:
model.reactions.get_by_id('FE2t').lower_bound = -1000.0
model.reactions.get_by_id('FE3t').lower_bound = -1000.0
model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000.0
model.reactions.get_by_id('EX_fe3_e').lower_bound = -1000.0

q9 biosynthesis  
HBZNPT_m: 4hbz_m + npdp_m --> 3nphb_m + ppi_m 9681

If mito, need to add  
TYRTAim: 34hpp_m + glu__L_m <=> akg_m + tyr__L_m GOT2 or TAT  
34HPLFM: 34hpp_m + h_m + nadh_m --> 34hpl_m + nad_m ?  
T4HCINNMFM: 34hpl_m --> T4hcinnm_m + h2o_m ?  
COUCOAFm: T4hcinnm_m + atp_m + coa_m --> amp_m + coucoa_m + ppi_m 16635  
4HBZCOAFm: coa_m + coucoa_m + h2o_m + nad_m --> 4hbzcoa_m + accoa_m + h_m + nadh_m ?  
4HBZFm: 4hbzcoa_m + h2o_m --> 4hbz_m + coa_m + h_m ?

4CMCOAS: T4hcinnm_c + atp_c + coa_c --> amp_c + coucoa_c + ppi_c 11833 or 13700 or 14802 or 16129

Where is 34hpp coming from?  
(e4p + pyr -> skm -> chor -> pphn -> 34hpp)

Chorismate biosynthesis (via shikimate)  
DDPA: e4p_c + h2o_c + pep_c --> 2dda7p_c + pi_c 10602 or 12155 or 15534  
DHQS: 2dda7p_c --> 3dhq_c + pi_c 14252  
DHQTi: 3dhq_c --> 3dhsk_c + h2o_c 10572 or 14252  
SHK3Dr: 3dhsk_c + h_c + nadph_c <=> nadp_c + skm_c 14252  
SHKK: atp_c + skm_c --> adp_c + h_c + skm5p_c 13706 or 14252  
PSCVT: pep_c + skm5p_c <=> 3psme_c + pi_c 14252  
CHORS: 3psme_c --> chor_c + pi_c 14037

dehydroshikimate dehydratase (3-dehydroshikimate -> protocatechuate + h2o) is missing  
9106 paperblast to 3-dehydroshikimate dehydratase  
DHSKDH: 3dhsk_c → h2o_c + 34dhbz_c 9106  
protocatechuate is 34dhbz_c

Phenylalanine biosynthesis (from chorismate via prephenate)  
CHORM: chor_c --> pphn_c 14195 or 16225 or 9704  
ADCS: chor_c + gln__L_c --> 4adcho_c + glu__L_c 13669  
ADCL: 4adcho_c --> 4abz_c + h_c + pyr_c 12476  
PPNDH: h_c + pphn_c --> co2_c + h2o_c + phpyr_c 14195  
PHETA1: akg_c + phe__L_c <=> glu__L_c + phpyr_c 14908 or 15839  
PPYRDC: h_c + phpyr_c --> co2_c + pacald_c 15791  
ALCD25xi: h_c + nadh_c + pacald_c --> 2phetoh_c + nad_c 14108 or 14109 or 15438  
ALCD25yi: h_c + nadph_c + pacald_c --> 2phetoh_c + nadp_c 10029 or 13554 or 13562 or 13947  
ALDD19x_P: h2o_c + nadp_c + pacald_c --> 2.0 h_c + nadph_c + pac_c 12042 or 13426 or 16323  
ALDD19xr: h2o_c + nad_c + pacald_c <=> 2.0 h_c + nadh_c + pac_c 12042 or 13426 or 16323  

16462 phenylalanine ammonia-lyase  

Tyrosine biosynthesis (from prephenate)  
PPND: nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c 14252 or 16225  
PPND2: nadp_c + pphn_c --> 34hpp_c + co2_c + nadph_c 16225  
TYRTA: akg_c + tyr__L_c <=> 34hpp_c + glu__L_c 14908 or 15839 or 16065

In [46]:
# 10602 ARO4 cyto, 12155 ARO3 mito, 15534 mito
model.reactions.get_by_id('DDPA').gene_reaction_rule = '10602'
model.reactions.get_by_id('DDPAm').gene_reaction_rule = '12155 or 15534'
# 14252 is ARO1 pentafunctional protein, but separate enzymes exist in rhodo
# Separate enzymes look like they are in mito, create mito reactions? keep cyto for now
# PPND is not catalyzed by 14252 ARO1, PPND2 is correct by 16225 TYR1 prephenate dehydrogenase (NADP+)
model.remove_reactions(['PPND'], remove_orphans=True)
# 13706 mito shikimate kinase / 3-dehydroquinate dehydratase type I (two functions), fitness defect in cou and fer
# SHKK okay, add 13706 to DHQTi
# 10572 looks like a mito aroQ 3-dehydroquinate dehydratase
model.reactions.get_by_id('DHQTi').gene_reaction_rule = '10572 or 13706 or 14252'
# 13709 shikimate / quinate 5-dehydrogenase (5' alternate splicing?), KEGG matches are quinate dehydrogenase
# Add QUINDH from eco, and set genes to 13709
r = eco.reactions.get_by_id('QUINDH').copy()
r.gene_reaction_rule = '13709'
model.add_reactions([r])
# 13710 (sigP) shikimate / quinate 5-dehydrogenase, but coverage is < 50%, leave it out for now
# shikimate kinase makes shikimate 3-phosphate, replace skm5p with skm3p
model.metabolites.get_by_id('skm5p_c').id = 'skm3p_c'
model.metabolites.get_by_id('skm3p_c').name = 'Shikimate 3-phosphate'
# Add DHSKDH from ppu, and set genes to 9106
r = ppu.reactions.get_by_id('DHSKDH').copy()
r.gene_reaction_rule = '9106'
model.add_reactions([r])
# 14195 prephenate dehydratase only (not bifunctional chorismate mutase/prephenate dehydratase), PPNDH correct
# 16225 prephenate dehydrogenase (NADP+), PPND2 correct
# 9704 chorismate mutase, CHORM
model.reactions.get_by_id('CHORM').gene_reaction_rule = '9704'
# 13669 Para-aminobenzoate (PABA) synthase ABZ1, ADCS correct
# 12476 ABZ2, ADCL is correct
# 4-aminobenzoate can be a precursor to ubiquinone biosynthesis -> add reactions

In [47]:
temp = ['10602','12155','15534','14252','10572','13706','13709','13710','14037','9106',
        '14195','16225','9704','13669','12476','14908','15839','15791',
        '14108','14109','15438','10029','13554','13562','13947','12042','13426','16323',
        '15434','16462','13444','13445']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10602,"K01626: E2.5.1.54, aroF, aroG, aroH; 3-deoxy-7...",,"ARO3,ARO4",,ARO4,,Essential,"cyto 22.5, cyto_mito 12, nucl 2",EKA*
12155,"K01626: E2.5.1.54, aroF, aroG, aroH; 3-deoxy-7...",,"ARO3,ARO4",,ARO3,,Not Essential,"mito 20, cyto 6.5, cyto_nucl 4",PAA*
15534,"K01626: E2.5.1.54, aroF, aroG, aroH; 3-deoxy-7...",,,,,,Not Essential,"mito 10, cyto_nucl 8, nucl 7.5, cyto 7.5",VKQ*
14252,K13830: ARO1; pentafunctional AROM polypeptide,,ARO1,,ARO1,,Essential,"cyto 12.5, cyto_nucl 7.5, plas 3, extr 3, E.R....",RDN*
10572,"K03786: aroQ, qutE; 3-dehydroquinate dehydrata...",,,,,,Not Essential,"mito 16, cyto 10.5, cyto_nucl 6",KQK*
13706,"K00891: E2.7.1.71, aroK, aroL; shikimate kinase",,,,ARO1,,Not Essential,"mito 15, cyto 5.5, cyto_nucl 5.5, nucl 4.5",RMG*
13709,KOG0692: Pentafunctional AROM protein,,,,ARO1,,Not Essential,"cyto 15, cyto_nucl 10.5, mito 5, nucl 4",PFE*
13710,KOG0692: Pentafunctional AROM protein,S,,,ARO1,,Not Essential,"mito 10, cyto 6, extr 5, nucl 3, plas 3, cyto_...",IGS*
14037,K01736: aroC; chorismate synthase,,ARO2,,ARO2,,Essential,"cyto 15.5, cyto_nucl 9, mito 7, pero 2",CSV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
12155,-0.310447,0.288119,0.166192,0.156131,-1.09158,0.22266,0.616564,-0.0241481,-0.33732,-0.126719,-0.268415
15534,-0.124421,-0.387767,-0.320253,-0.0883859,0.059986,0.319192,-0.181263,-0.91462,0.160326,-0.138136,-0.137485
10572,-0.119734,-0.168895,-0.116319,0.0274772,-0.107331,0.105408,-0.00101536,0.251161,0.256098,0.183985,-0.411911
13706,0.00469469,0.241651,0.155219,-0.0508848,-1.80215,-0.994622,-0.320292,-1.71794,-0.409246,-0.139143,-0.046574
13709,0.128287,-0.0819031,0.241922,0.225473,0.355881,0.0161442,0.147499,-0.0217116,-0.0521156,0.00706958,-0.0412034
13710,0.0339886,-0.0967401,0.149373,0.0257554,0.117254,-0.00539465,-0.0998042,-0.246703,0.168151,-0.0269269,0.00424615
9106,-0.0566865,-0.376082,-0.0190414,-0.191385,0.00924402,-0.172002,-0.0708188,-0.207445,0.0876317,0.27866,0.0112956
14195,-0.355315,-0.506645,-0.987004,0.022568,-3.17046,-0.42575,-3.15518,-2.42223,-4.24174,-3.07117,-0.325324
13669,0.0145212,-0.149855,-0.0347267,0.0314421,0.0858744,0.127857,0.079359,-0.172931,0.00296095,0.0207803,-0.20695


In [48]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

DDPA: e4p_c + h2o_c + pep_c --> 2dda7p_c + pi_c 10602

DDPAm: e4p_m + h2o_m + pep_m --> 2dda7p_m + pi_m 12155 or 15534

DDPAm: e4p_m + h2o_m + pep_m --> 2dda7p_m + pi_m 12155 or 15534

DHQS: 2dda7p_c --> 3dhq_c + pi_c 14252
DHQTi: 3dhq_c --> 3dhsk_c + h2o_c 10572 or 13706 or 14252
PSCVT: pep_c + skm3p_c <=> 3psme_c + pi_c 14252
SHK3Dr: 3dhsk_c + h_c + nadph_c <=> nadp_c + skm_c 14252
SHKK: atp_c + skm_c --> adp_c + h_c + skm3p_c 13706 or 14252

DHQTi: 3dhq_c --> 3dhsk_c + h2o_c 10572 or 13706 or 14252

DHQTi: 3dhq_c --> 3dhsk_c + h2o_c 10572 or 13706 or 14252
SHKK: atp_c + skm_c --> adp_c + h_c + skm3p_c 13706 or 14252

QUINDH: nad_c + quin_c --> 3dhq_c + 2.0 h_c + nadh_c 13709

13710 no reactions

CHORS: 3psme_c --> chor_c + pi_c 14037

DHSKDH: 3dhsk_c --> 34dhbz_c + h2o_c 9106

PPNDH: h_c + pphn_c --> co2_c + h2o_c + phpyr_c 14195

PPND2: nadp_c + pphn_c --> 34hpp_c + co2_c + nadph_c 16225

CHORM: chor_c --> pphn_c 9704

ADCS: chor_c + gln__L_c --> 4adcho_c + glu__L_c 13669

ADCL: 4a

15434 is glutathione S-transferase  
16462 Phenylalanine/tyrosine amino-lyase  
https://www.ncbi.nlm.nih.gov/pubmed/25911487  
Add PAL and TAL reactions  
13444, 13445 caffeoyl-CoA O-methyltransferase?

In [49]:
r = model.reactions.get_by_id('HISDr')
r.id = 'PAL'
r.name = 'Phenylalanine ammonia-lyase'
r.gene_reaction_rule = '16462'
model.add_metabolites([eco.metabolites.get_by_id('cinnm_c').copy()])
r.add_metabolites({'his__L_c': 1.0, 'urcan_c': -1.0, 'phe__L_c': -1.0, 'cinnm_c': 1.0})

r = model.reactions.get_by_id('PAL').copy()
r.id = 'TAL'
r.name = 'Tyrosine ammonia-lyase'
r.gene_reaction_rule = '16462'
model.add_reactions([r])
r.add_metabolites({'phe__L_c': 1.0, 'cinnm_c': -1.0, 'tyr__L_c': -1.0, 'T4hcinnm_c': 1.0})

In [50]:
m1 = ppu.metabolites.get_by_id('caffcoa_c').copy()
m2 = ppu.metabolites.get_by_id('ferulcoa_c').copy()
model.add_metabolites([m1,m2])
r = model.reactions.get_by_id('34DHPHAMT').copy()
r.id = 'CCOAOMT'
r.name = 'Caffeoyl-CoA O-methyltransferase'
r.gene_reaction_rule = '13444 or 13445'
model.add_reactions([r])
r.add_metabolites({'34dhpha_c': 1.0, 'homoval_c': -1.0, 'caffcoa_c': -1.0, 'ferulcoa_c': 1.0})

In [51]:
for r in sorted(model.metabolites.get_by_id('T4hcinnm_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('T4hcinnm_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ferulcoa_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

4CMCOAS: T4hcinnm_c + atp_c + coa_c --> amp_c + coucoa_c + ppi_c 11833 or 13700 or 14802 or 16129
TAL: tyr__L_c --> T4hcinnm_c + nh4_c 16462

COUCOAFm: T4hcinnm_m + atp_m + coa_m --> amp_m + coucoa_m + ppi_m 16635

CCOAOMT: amet_c + caffcoa_c --> ahcys_c + ferulcoa_c + h_c 13444 or 13445


q9 biosynthesis  

If coumarate is coming from 34hpp instead of TAL,  
TYRTAim: 34hpp_m + glu__L_m <=> akg_m + tyr__L_m GOT2 or TAT (this step is by mito aminotransferase)  
34HPLFM: 34hpp_m + h_m + nadh_m --> 34hpl_m + nad_m ?  
T4HCINNMFM: 34hpl_m --> T4hcinnm_m + h2o_m ?  

S.cer https://www.nature.com/articles/nbt.3683.pdf  
tyr__L -> 34hpp by Aro9 (no ortholog, ARO8 could do it in cyto)  
34hpp -> 34hpacald by Aro10 (no ortholog, PDC could do it in cyto)  
34hpacald -> T4hcinnm by Aim18? (16521)

C. sporogenes https://www.nature.com/articles/nature24661.pdf  
tyr__L -> 34hpp by aat (many)  
34hpp -> 34hpac by porA pyruvate:ferredoxin dehydrogenase (no ortholog)  
34hpp -> 34hpl by fldH aromatic 2-oxoacid reductase (blast to 9085 D-3-phosphoglycerate dehydrogenase)  
34hpl -> 34hpl-CoA by fldA (9923 succinate---hydroxymethylglutarate CoA-transferase)  
34hpl-CoA -> T4hcinnm-CoA by fldBC (no ortholog)  
34hpl -> 3-hydroxyphenylpropanoate by acdA (14070 ACADSB / 12570 ACADM / 10012 IVDe)

no complete pathway from 34hpp to coumarate except 34hpp -> tyr -> T4hcinnm  

In [52]:
Annotation.loc[['16521','9085','9923','14070','12570','10012']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16521,"HMMPfam:Chalcone-flavanone isomerase:PF02431,S...",A,AIM18,,,,Not Essential,mito 26,GAA*
9085,"K00058: serA, PHGDH; D-3-phosphoglycerate dehy...",,"SER3,SER33",PHGDH,SER33,PHGDH,Not Essential,"extr 12, mito 10, pero 2, nucl 1, cyto 1, cyto...",MLF*
9923,K18703: SUGCT; succinate---hydroxymethylglutar...,S,,SUGCT,,SUGCT,Not Essential,"mito 26.5, cyto_mito 14",GVV*
14070,K09478: ACADSB; short/branched chain acyl-CoA ...,,,ACADSB,,ACADS,Not Essential,"mito 26, cyto_mito 14.5",EYS*
12570,"K00249: ACADM, acd; acyl-CoA dehydrogenase",,,ACADM,,ACADM,Not Essential,mito 26,DYA*
10012,"K00253: IVD, ivd; isovaleryl-CoA dehydrogenase",S,,IVD,,IVDe,Not Essential,"mito 25.5, cyto_mito 14",GAL*


In [53]:
# 4HB can be made from coumarate, or para-aminobenzoate can substitute 4HB for ubiquinone synthesis
r1 = sce.reactions.get_by_id('EX_4abz_e').copy()
r2 = sce.reactions.get_by_id('4ABZt').copy()
r3 = sce.reactions.get_by_id('4ABZtm').copy()
model.add_reactions([r1,r2,r3])

m1 = model.metabolites.get_by_id('3nphb_m').copy()
m1.id = '3npab_m'
m1.name = '3-Nonaprenyl-4-aminobenzoate'
m1.formula = 'C52H78NO2'
m2 = model.metabolites.get_by_id('3npdhb_m').copy()
m2.id = '3npahb_m'
m2.name = '3-Nonaprenyl-4-amino-5-hydroxybenzoate'
m2.formula = 'C52H78NO3'
model.add_metabolites([m1,m2])

r = model.reactions.get_by_id('HBZNPT_m').copy()
r.id = 'ABZNPT_m'
r.name = 'Aminobenzoate Nonaprenyltransferase'
model.add_reactions([r])
r.add_metabolites({'4hbz_m': 1.0, '3nphb_m': -1.0, '4abz_m': -1.0, '3npab_m': 1.0})

r = model.reactions.get_by_id('3NPHBH2_m').copy()
r.id = '3NPABH_m'
r.name = 'Hydroxylation of 3-nonaaprenyl-4-aminobenzoate (NADP)'
model.add_reactions([r])
r.add_metabolites({'3nphb_m': 1.0, '3npdhb_m': -1.0, '3npab_m': -1.0, '3npahb_m': 1.0})

r = model.reactions.get_by_id('3NPHBH2_m').copy()
r.id = '3NPAHBDH_m'
r.name = 'Deamination and hydroxylation of 3-nonaaprenyl-4-amino-5-hydroxybenzoate (NADP)'
model.add_reactions([r])
r.add_metabolites({'3nphb_m': 1.0, '3npahb_m': -1.0, 'nadph_m': -1.0, 'h_m': -2.0, 'nadp_m': 1.0, 'nh4_m': 1.0})

In [54]:
# check if model can produce cofactors
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
with model:
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                #if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                if abs(v) > 1:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234278
coa_c 0.13547854502629483
fad_c 0.1358833457242419
gthrd_c 0.13632034044837316
hemeA_m 0.1352517764125231
nad_c 0.13561558122627718
nadp_c 0.1356003412933258
q9_m 0.13393271057829534
thf_c 0.1359721547056612
thmpp_c 0.13561761347616483
5mthf_c 0.13577122443665932
btn_m 0.0
	 btn_m -100.0
	 dad_5_m 100.0
lipopb_m 0.0
	 met__L_m 33.33333333333333
	 mettrna_m 33.33333333333333
	 fmettrna_m 33.33333333333333
	 2fe2s_m 33.33333333333333
	 lipoamp_m -100.0
	 lipopb_m -100.0
	 lipoate_m -100.0
mlthf_c 0.135755949504487
ptrc_c 0.13650642647367803
pydx5p_c 0.13653113901894637
spmd_c 0.1365774991663852


Replace amino acids with aminoacyl-tRNA for protein?  
Check aminoacyl-tRNA synthetase

In [55]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('tRNA synthetase')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8575,"K01892: HARS, hisS; histidyl-tRNA synthetase",,HTS1,"HARS,HARS2",HTS1,HARS,Essential,"mito 25.5, cyto_mito 14",GKK*
9167,"K01893: NARS, asnS; asparaginyl-tRNA synthetase",,DED81,NARS,DED81,NARS,Essential,"cyto 7.5, cysk 7, cyto_nucl 5.5, mito 5, pero ...",CTP*
9334,BLAST: probable GUS1-Glutamyl-tRNA synthetase ...,,,,,,Not Essential,"nucl 11, cyto_nucl 9, mito 7, cyto 5, extr 3",ALV*
9855,"K01883: CARS, cysS; cysteinyl-tRNA synthetase",,YNL247W,"CARS,CARS2",YNL247W,CARS,Essential,"cyto 13.5, cyto_mito 9.5, nucl 5, mito 4.5, pe...",QSA*
10079,"K01887: RARS, argS; arginyl-tRNA synthetase",,"YDR341C,MSR1","RARS2,RARS",YDR341C,RARS2,Essential,"cyto 20, cyto_nucl 12, mito 5",ERM*
10144,"K01869: LARS, leuS; leucyl-tRNA synthetase",,CDC60,LARS,CDC60,LARS,Essential,"cyto 13.5, cyto_mito 11, mito 7.5, nucl 4",YNI*
10246,"K01875: SARS, serS; seryl-tRNA synthetase",,SES1,SARS,SES1,SARS,Essential,"cyto 14.5, cyto_nucl 12, nucl 8.5, pero 2",QKH*
10753,"K01885: EARS, gltX; glutamyl-tRNA synthetase",,GUS1,,GUS1,EPRS,Essential,"cyto_mito 10.5, mito 10, cyto 9, nucl 5",INE*
11486,"K01881: PARS, proS; prolyl-tRNA synthetase",,YHR020W,EPRS,YHR020W,EPRS,Not Essential,"cyto 17, cyto_mito 11.833, cyto_nucl 9.833, mi...",RSY*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9334,-0.054738,-0.115217,-0.158594,0.0709991,-0.514091,-0.15432,-0.0958847,0.106688,-0.283772,-0.107636,0.0440375
9855,-0.127481,0.180375,0.178513,0.249167,1.66084,0.563794,0.0563758,0.586144,0.594192,0.61092,-0.179258
11486,-0.105912,-0.0575314,-0.199827,-0.0594206,0.190392,-0.00969138,-0.0508443,-0.138312,-0.210517,-0.183923,-0.0170305
11945,-0.475597,-0.0471994,-0.467938,0.159971,0.078547,-0.217404,-0.410717,0.0768835,-0.145783,0.330613,0.626294
13380,0.212952,0.48865,0.134225,0.480758,-0.595638,0.105697,-0.289997,-0.915948,0.122397,-0.08248,-0.750243
15848,-1.02181,-0.967317,-1.48895,-0.786704,-0.959296,-1.31833,-0.0313898,0.319973,0.277672,0.15802,0.360635


In [56]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

HISTRS: atp_c + his__L_c + trnahis_c --> amp_c + histrna_c + ppi_c 8575
HISTRSm: atp_m + his__L_m + trnahis_m --> amp_m + histrna_m + ppi_m 8575

ASNTRS: asn__L_c + atp_c + trnaasn_c --> amp_c + asntrna_c + ppi_c 9167

9334 no reactions

CYSTRS: atp_c + cys__L_c + trnacys_c --> amp_c + cystrna_c + ppi_c 9855

ARGTRS: arg__L_c + atp_c + trnaarg_c --> amp_c + argtrna_c + ppi_c 10079
ARGTRSm: arg__L_m + atp_m + trnaarg_m --> amp_m + argtrna_m + ppi_m 10079

LEUTRS: atp_c + leu__L_c + trnaleu_c --> amp_c + leutrna_c + ppi_c 10144
LEUTRSm: atp_m + leu__L_m + trnaleu_m --> amp_m + leutrna_m + ppi_m 10144

SECYSTL: atp_m + ser__L_m + trnasecys_m --> amp_m + ppi_m + sertrna_sec_m 10246 or 14225
SERTRS: atp_c + ser__L_c + trnaser_c --> amp_c + ppi_c + sertrna_c 10246 or 14225

GLUTRS: atp_c + glu__L_c + trnaglu_c --> amp_c + glutrna_c + ppi_c 10753 or 14565

PROTRS: atp_c + pro__L_c + trnapro_c --> amp_c + ppi_c + protrna_c 11486

AP4AS: 2.0 atp_c + h_c --> ap4a_c + ppi_c 11945
LYSTRS: atp_c + 

In [57]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.endswith('trna_c'):
        for r in sorted(m.reactions, key=lambda x: x.id):
            print(r, r.gene_reaction_rule)
        if m.id.replace('_c','_m') in model.metabolites:
            for r in sorted(model.metabolites.get_by_id(m.id.replace('_c','_m')).reactions, key=lambda x: x.id):
                print(r, r.gene_reaction_rule)
        print()

ALATRS: ala__L_c + atp_c + trnaala_c --> alatrna_c + amp_c + ppi_c 11973
ALATL: ala__L_m + atp_m + trnaala_m --> alatrna_m + amp_m + ppi_m 11973

ARGTRS: arg__L_c + atp_c + trnaarg_c --> amp_c + argtrna_c + ppi_c 10079
ARGTRSm: arg__L_m + atp_m + trnaarg_m --> amp_m + argtrna_m + ppi_m 10079

ASNTRS: asn__L_c + atp_c + trnaasn_c --> amp_c + asntrna_c + ppi_c 9167
ASNTRSm: asn__L_m + atp_m + trnaasn_m --> amp_m + asntrna_m + ppi_m 12123

ASPTRS: asp__L_c + atp_c + trnaasp_c --> amp_c + asptrna_c + ppi_c 14258
ASPTRSm: asp__L_m + atp_m + trnaasp_m --> amp_m + asptrna_m + ppi_m 14258

CYSTRS: atp_c + cys__L_c + trnacys_c --> amp_c + cystrna_c + ppi_c 9855

FMETTRS: 10fthf_c + mettrna_c --> fmettrna_c + h_c + thf_c 11899
FMETTRSm: 10fthf_m + mettrna_m --> fmettrna_m + h_m + thf_m 11899

GLNTRS: atp_c + gln__L_c + trnagln_c --> amp_c + glntrna_c + ppi_c 13380

GLUTRS: atp_c + glu__L_c + trnaglu_c --> amp_c + glutrna_c + ppi_c 10753 or 14565
GLUTRSm: atp_m + glu__L_m + trnaglu_m --> amp_m + 

In [58]:
for r in sorted(model.metabolites.get_by_id('mettrna_c').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('mettrna_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

FMETTRS: 10fthf_c + mettrna_c --> fmettrna_c + h_c + thf_c 11899
METTRS: atp_c + met__L_c + trnamet_c --> amp_c + mettrna_c + ppi_c 13140 or 15518

FMETTRSm: 10fthf_m + mettrna_m --> fmettrna_m + h_m + thf_m 11899
METTRSm: atp_m + met__L_m + trnamet_m --> amp_m + mettrna_m + ppi_m 15518


In [59]:
model.reactions.get_by_id('GLUTRS').gene_reaction_rule = '10753'
model.reactions.get_by_id('METTRS').gene_reaction_rule = '13140'
model.reactions.get_by_id('SERTRS').gene_reaction_rule = '10246'
model.reactions.get_by_id('TRPTRS').gene_reaction_rule = '14709'
model.reactions.get_by_id('TYRTRS').gene_reaction_rule = '16641'

# Make mito seryl-tRNA synthetase by 14225
r = model.reactions.get_by_id('SERTRS').copy()
r.id = 'SERTRSm'
r.gene_reaction_rule = '14225'
model.add_reactions([r])
for m in r.metabolites:
    if not m.id.replace('_c','_m') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_m')
        m2.compartment = 'm'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})

# AP4AS by E. coli lysU, but not by KRS1 -> remove AP4AS
# 11973 cyto, no ALATL in iMM904 -> remove ALATL
# 11899 mito -> remove FMETTRS
# no selenocysteine synthase in annotation -> remove SECYSTL
model.remove_reactions(['AP4AS','ALATL','FMETTRS','SECYSTL'], remove_orphans=True)

In [60]:
# check if model can produce cofactors
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
with model:
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                #if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                if abs(v) > 1:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234173
coa_c 0.13547854502629292
fad_c 0.13588334572424138
gthrd_c 0.13632034044837502
hemeA_m 0.13525177641252495
nad_c 0.13561558122627657
nadp_c 0.13560034129332058
q9_m 0.13393271057829698
thf_c 0.13597215470565893
thmpp_c 0.13561761347616552
5mthf_c 0.13577122443665735
btn_m 0.0
	 adp_m 100.0
	 atp_m 100.0
	 amp_m 100.0
	 fad_m 100.0
	 fadh2_m 100.0
	 met__L_m 100.0
	 mettrna_m 100.0
	 fmettrna_m 100.0
	 nmn_m -100.0
	 dnad_m 100.0
	 btn_m -100.0
	 lipoamp_m -100.0
	 lipopb_m -200.0
	 lipoate_m -200.0
lipopb_m 0.0
	 adp_m 49.99999999999999
	 atp_m 49.99999999999999
	 amp_m 49.99999999999999
	 fad_m 49.99999999999999
	 fadh2_m 49.99999999999999
	 met__L_m 49.99999999999999
	 mettrna_m 49.99999999999999
	 fmettrna_m 49.99999999999999
	 nmn_m -49.99999999999999
	 dnad_m 49.99999999999999
	 btn_m -49.99999999999999
	 lipoamp_m -49.99999999999999
	 lipopb_m -99.99999999999999
	 lipoate_m -99.99999999999999
mlthf_c 0.13575594950448788
ptrc_c 0.13650642647368216
pydx5

In [61]:
for r in sorted(model.metabolites.get_by_id('dann_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
for r in sorted(model.metabolites.get_by_id('btn_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('lipoamp_m').reactions, key=lambda x: x.id):
    print(r, r.gene_reaction_rule)

AMAOTrm: 8aonn_m + amet_m <=> amob_m + dann_m 12731
DBTSm: atp_m + co2_m + dann_m <=> adp_m + dtbt_m + 3.0 h_m + pi_m 12731
BACCLm: atp_m + btn_m + h_m --> btamp_m + ppi_m 16404
BTS5m: 2fe2s_m + amet_m + dtbt_m --> 2fe1s_m + btn_m + dad_5_m + h_m + met__L_m 15908

LIPAMPLm: lipoamp_m --> amp_m + lipopb_m 13899
LIPATPTm: atp_m + lipoate_m --> lipoamp_m + ppi_m 13899


In [62]:
temp = ['12731','15908','16404','13899']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12731,K19562: BIO3-BIO1; bifunctional dethiobiotin s...,,,,BIO3,,Essential,"mito 11, extr 10, cyto 3, pero 2",GAS*
15908,K01012: bioB; biotin synthase,,BIO2,,BIO2,,Essential,"mito 26.5, cyto_mito 14",VAA*
16404,K01942: HLCS; biotin--protein ligase,,BPL1,HLCS,BPL1,HLCS,Essential,"extr 12, cyto 6, cyto_nucl 5.5, mito 4, nucl 3",KSG*
13899,"K03800: lplA, lplJ; lipoate---protein ligase",,AIM22,LIPT1,AIM22,LIPT1,Essential,"mito 20, nucl 4, cyto 1, plas 1, pero 1, cyto_...",GEM*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16404,0.224466,-0.772008,-0.234875,-0.5554,-0.0770478,0.701184,0.14778,-0.208173,-1.19924,-0.980346,0.058461


In [63]:
cofactors = ['btn_m','lipopb_m']

with model:
    r = model.reactions.get_by_id('DM_aacald_c').copy()
    r.id = 'SRC_8aonn_m'
    r.name = 'Source needed to allow 8-amino-7-oxononanoate to enter system'
    model.add_reactions([r])
    r.add_metabolites({'aacald_c': 1.0, '8aonn_m': 1.0})

    r = model.reactions.get_by_id('DM_aacald_c').copy()
    r.id = 'DM_amob_m'
    r.name = 'Demand needed to allow S-Adenosyl-4-methylthio-2-oxobutanoate to leave system'
    model.add_reactions([r])
    r.add_metabolites({'aacald_c': 1.0, 'amob_m': -1.0})

    r = model.reactions.get_by_id('DM_aacald_c').copy()
    r.id = 'DM_dad_5_m'
    r.name = 'Demand needed to allow 5-Deoxyadenosine to leave system'
    model.add_reactions([r])
    r.add_metabolites({'aacald_c': 1.0, 'dad_5_m': -1.0})
    
    r1 = hsa2.reactions.get_by_id('METtm').copy()
    r1.lower_bound = -1000.0
    r2 = hsa2.reactions.get_by_id('r1437').copy()
    r2.id = 'CYStm'
    r2.lower_bound = -1000.0
    model.add_reactions([r1,r2])

    r = model.reactions.get_by_id('DM_aacald_c').copy()
    r.id = 'SRC_lipoate_m'
    r.name = 'Source needed to allow lipoate to enter system'
    model.add_reactions([r])
    r.add_metabolites({'aacald_c': 1.0, 'lipoate_m': 1.0})

    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                #if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                if abs(v) > 1:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

btn_m 0.13491103213722255
lipopb_m 0.13655173964275558


In [64]:
r = model.reactions.get_by_id('DM_aacald_c').copy()
r.id = 'SRC_8aonn_m'
r.name = 'Source needed to allow 8-amino-7-oxononanoate to enter system'
model.add_reactions([r])
r.add_metabolites({'aacald_c': 1.0, '8aonn_m': 1.0})

r = model.reactions.get_by_id('DM_aacald_c').copy()
r.id = 'DM_amob_m'
r.name = 'Demand needed to allow S-Adenosyl-4-methylthio-2-oxobutanoate to leave system'
model.add_reactions([r])
r.add_metabolites({'aacald_c': 1.0, 'amob_m': -1.0})

r = model.reactions.get_by_id('DM_aacald_c').copy()
r.id = 'DM_dad_5_m'
r.name = 'Demand needed to allow 5-Deoxyadenosine to leave system'
model.add_reactions([r])
r.add_metabolites({'aacald_c': 1.0, 'dad_5_m': -1.0})

r1 = hsa2.reactions.get_by_id('METtm').copy()
r1.lower_bound = -1000.0
r2 = hsa2.reactions.get_by_id('r1437').copy()
r2.id = 'CYStm'
r2.lower_bound = -1000.0
model.add_reactions([r1,r2])

r = model.reactions.get_by_id('DM_aacald_c').copy()
r.id = 'SRC_lipoate_m'
r.name = 'Source needed to allow lipoate to enter system'
model.add_reactions([r])
r.add_metabolites({'aacald_c': 1.0, 'lipoate_m': 1.0})

In [65]:
# check if model can produce cofactors
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
with model:
    for x in cofactors:
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-2})
        sol = model.optimize()
        print(x, sol.objective_value)
        if abs(sol.objective_value) < 1e-6:
            for k, v in sol.shadow_prices.items():
                #if v and k in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]:
                if abs(v) > 1:
                    print('\t',k, v)
        model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: 1e-2})

camp_c 0.13593743393234295
coa_c 0.13547854502629367
fad_c 0.1358833457242399
gthrd_c 0.1363203404483766
hemeA_m 0.13525177641251918
nad_c 0.13561558122627665
nadp_c 0.13560034129332127
q9_m 0.1339327105783163
thf_c 0.13597215470565932
thmpp_c 0.13561761347616408
5mthf_c 0.13577122443665915
btn_m 0.1349110321372241
lipopb_m 0.13655173964275663
mlthf_c 0.13575594950448663
ptrc_c 0.1365064264736743
pydx5p_c 0.1365311390189447
spmd_c 0.13657749916638584


In [66]:
cofactors = ['camp_c','coa_c','fad_c','gthrd_c','hemeA_m','nad_c','nadp_c','q9_m','thf_c','thmpp_c',
             '5mthf_c','btn_m','lipopb_m','mlthf_c','ptrc_c','pydx5p_c','spmd_c']
[x for x in cofactors if x in [m.id for m in model.reactions.get_by_id('BIOMASS_RT').metabolites]]

[]

In [67]:
for x in cofactors:
    model.reactions.get_by_id('BIOMASS_RT').add_metabolites({x: -1e-6})

In [68]:
sol = model.optimize()
print(sol.objective_value)

0.13657612542330205


In [69]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1127
2117
1956


Name,R. toruloides
Memory address,0x0102adc2e10
Number of metabolites,1956
Number of reactions,2117
Number of groups,0
Objective expression,1.0*BIOMASS_RT - 1.0*BIOMASS_RT_reverse_2b3e0
Compartments,"c, x, m, e, r, v, n, g, d"


In [70]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

10885
12640
14615
14700
14875
15434
8666

hdc2ea_c
hxdcal_c
hxdceal_c
pristanal_c
urcan_c


In [71]:
cobra.manipulation.remove_genes(model, [x for x in model.genes if not x.reactions])
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

In [72]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
print(len(set([m.id.rsplit('_',1)[0] for m in model.metabolites])))
print(len(model.compartments))
model

1120
2117
1951
1227
9


Name,R. toruloides
Memory address,0x0102adc2e10
Number of metabolites,1951
Number of reactions,2117
Number of groups,0
Objective expression,1.0*BIOMASS_RT - 1.0*BIOMASS_RT_reverse_2b3e0
Compartments,"c, x, m, e, r, v, n, g, d"


In [73]:
cobra.io.save_json_model(model, "IFO0880_GPR_2b.json")

In [74]:
model_old = cobra.io.load_json_model("IFO0880_GPR_2a.json")
model_new = cobra.io.load_json_model("IFO0880_GPR_2b.json")

In [75]:
print('Removed reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r not in model_new.reactions:
        print(r)

Removed reactions

3HPADHi: 3hppnl_c + h2o_c + nad_c --> 3hpp_c + 2.0 h_c + nadh_c
3M4HDXPAC: 3mox4hpac_c + h2o_c + nad_c <=> 2.0 h_c + homoval_c + nadh_c
3MOX4HOXPGALDOX: 3m4hpga_c + h2o_c + nad_c --> 3mox4hoxm_c + 2.0 h_c + nadh_c
3MOX4HOXPGALDOX_NADP: 3m4hpga_c + h2o_c + nadp_c <=> 3mox4hoxm_c + 2.0 h_c + nadph_c
5HOXINDACTOX: 5hoxindact_c + h2o_c + nad_c --> 5hoxindoa_c + 2.0 h_c + nadh_c
5HOXINDACTOXm: 5hoxindact_m + h2o_m + nad_m --> 5hoxindoa_m + 2.0 h_m + nadh_m
ABOR: 4abutn_m + h2o_m + nadp_m --> 4abut_m + 2.0 h_m + nadph_m
ABUTDm: 4abutn_m + h2o_m + nad_m --> 4abut_m + 2.0 h_m + nadh_m
AKP1: ahdt_c + 3.0 h2o_c --> dhnpt_c + 2.0 h_c + 3.0 pi_c
ALATL: ala__L_m + atp_m + trnaala_m --> alatrna_m + amp_m + ppi_m
ALDD20xm: h2o_m + id3acald_m + nad_m --> 2.0 h_m + ind3ac_m + nadh_m
ALDD20ym: h2o_m + id3acald_m + nadp_m --> 2.0 h_m + ind3ac_m + nadph_m
ALDD21: h2o_c + nad_c + pristanal_c --> 2.0 h_c + nadh_c + prist_c
AP4AS: 2.0 atp_c + h_c --> ap4a_c + ppi_c
BAMPPALDOXm: bamppald_m 

In [76]:
print('Updated reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r in model_new.reactions:
        r2 = model_new.reactions.get_by_id(r.id)
        if (r.name == r2.name and r.reaction == r2.reaction and r.gene_reaction_rule == r2.gene_reaction_rule and
            r.lower_bound == r2.lower_bound and r.upper_bound == r2.upper_bound):
            pass
        else:
            print('Old', r, r.gene_reaction_rule)
            print('New', r2, r2.gene_reaction_rule)
            print()

Updated reactions

Old 2DOXG6PP: 2doxg6p_c + h2o_c --> 2dglc_c + pi_c 13413 or 8460 or 8576
New 2DOXG6PP: 2doxg6p_c + h2o_c --> 2dglc_c + pi_c 13413

Old 34DHALDD: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323
New 34DHALDD: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c 12042

Old 34DHPLACOX_NADP: 34dhpac_c + h2o_c + nadp_c <=> 34dhpha_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323
New 34DHPLACOX_NADP: 34dhpac_c + h2o_c + nadp_c <=> 34dhpha_c + 2.0 h_c + nadph_c 12042

Old 4HOXPACDOX_NADP: 4hoxpacd_c + h2o_c + nadp_c <=> 4hphac_c + 2.0 h_c + nadph_c 12042 or 13426 or 16323
New 4HOXPACDOX_NADP: 4hoxpacd_c + h2o_c + nadp_c <=> 4hphac_c + 2.0 h_c + nadph_c 12042

Old ABUTD: 4abutn_c + h2o_c + nad_c --> 4abut_c + 2.0 h_c + nadh_c 12042 or 13426 or 16323
New ABUTD: 4abutn_c + h2o_c + nad_c --> 4abut_c + 2.0 h_c + nadh_c 12042

Old ACP1e: fmn_e + h2o_e --> pi_e + ribflv_e 10885 or 13856 or 13935
New ACP1e: fmn_e + h2o_e --> pi_e + ribflv_e 1385

In [77]:
print('Added reactions\n')
for r in sorted(model_new.reactions, key=lambda x: x.id):
    if r not in model_old.reactions:
        print(r)

Added reactions

3NPABH_m: 3npab_m + h_m + nadph_m + o2_m --> 3npahb_m + h2o_m + nadp_m
3NPAHBDH_m: 3npahb_m + 3.0 h_m + 2.0 nadph_m + o2_m --> 3npdhb_m + h2o_m + 2.0 nadp_m + nh4_m
4ABZt: 4abz_c <=> 4abz_e
4ABZtm: 4abz_c <=> 4abz_m
4HBALDDm: 4hbald_m + h2o_m + nad_m --> 4hbz_m + 2.0 h_m + nadh_m
ABZNPT_m: 4abz_m + npdp_m --> 3npab_m + ppi_m
ALDD16er: h2o_r + hxdcal_r + nad_r --> 2.0 h_r + hdca_r + nadh_r
ALDD21er: h2o_r + nad_r + pristanal_r --> 2.0 h_r + nadh_r + prist_r
ALDD3C161er: h2o_r + hxdceal_r + nad_r --> 2.0 h_r + hdc2ea_r + nadh_r
CCOAOMT: amet_c + caffcoa_c --> ahcys_c + ferulcoa_c + h_c
CYStm: cys__L_c <=> cys__L_m
DHFRim: dhf_m + h_m + nadph_m --> nadp_m + thf_m
DHNPA2r: dhnpt_c <=> 6hmhpt_c + gcald_c
DHSKDH: 3dhsk_c --> 34dhbz_c + h2o_c
DM_4oglu_c: 4oglu_c --> 
DM_amob_m: amob_m --> 
DM_dad_5_m: dad_5_m --> 
EX_4abz_e: 4abz_e --> 
EX_glx_e: glx_e --> 
EX_glyclt_e: glyclt_e --> 
FAAH: h2o_c + ocdcead_c --> nh4_c + ocdcea_c
GLXt: glx_c --> glx_e
GLXtm: glx_c <=> glx_m
GLX